In [1]:
# step1_eda_lg_fault.py
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import signal

# === CONFIG ===
CSV_PATH = "merged_dataset.csv"   # change if your file path differs
OUT_DIR = "eda_outputs"
os.makedirs(OUT_DIR, exist_ok=True)

# === 1. Load ===
print("Loading:", CSV_PATH)
df = pd.read_csv(CSV_PATH)
print("Loaded dataframe.")

# === 2. Basic info ===
print("\n=== Basic info ===")
print("Shape:", df.shape)
print("Columns:", df.columns.tolist())
print("\nDtypes:\n", df.dtypes)
print("\nFirst 5 rows:\n", df.head().to_string(index=False))
print("\nLast 5 rows:\n", df.tail().to_string(index=False))

# Missing values
print("\nMissing values per column:")
print(df.isnull().sum())

# Unique labels for Fault
if "Fault" in df.columns:
    print("\nUnique Fault labels and counts:")
    print(df["Fault"].value_counts(dropna=False))
else:
    print("\nColumn 'Fault' not found in CSV. Please confirm column names.")

# === 3. Sampling interval estimation ===
if "t" in df.columns:
    t = df["t"].values.astype(float)
    dt = np.diff(t)
    # filter out zero or negative dt if any
    dt = dt[np.isfinite(dt) & (dt>0)]
    if len(dt) == 0:
        print("\nNo positive time differences found in 't' to estimate sampling interval.")
    else:
        print("\nSampling interval (seconds):")
        print(" count:", len(dt))
        print(" mean: {:.6f}".format(np.mean(dt)))
        print(" median: {:.6f}".format(np.median(dt)))
        print(" std: {:.6f}".format(np.std(dt)))
        # approximate sampling frequency
        fs_median = 1.0 / np.median(dt)
        print(" approx sampling frequency (Hz) from median dt: {:.3f} Hz".format(fs_median))
else:
    print("\nColumn 't' not found in CSV. Please confirm column names.")

# === 4. Quick plot: full time-series overview (stacked) ===
channels_v = [c for c in df.columns if c.lower().startswith("v")]
channels_i = [c for c in df.columns if c.lower().startswith("i")]
channels = channels_v + channels_i

if len(channels) == 0:
    print("\nNo Va/Vb/Vc/Ia/Ib/Ic columns detected by prefix. Detected columns:", df.columns.tolist())
else:
    fig, axs = plt.subplots(len(channels), 1, figsize=(14, 2.5*len(channels)), sharex=True)
    for ax, ch in zip(axs, channels):
        ax.plot(df["t"], df[ch], linewidth=0.5)
        ax.set_ylabel(ch)
        ax.grid(True, linewidth=0.3)
    axs[-1].set_xlabel("t (s)")
    fig.suptitle("Waveform overview")
    out1 = os.path.join(OUT_DIR, "waveform_overview.png")
    plt.tight_layout(rect=[0,0,1,0.96])
    plt.savefig(out1, dpi=200)
    plt.close(fig)
    print("\nSaved waveform overview to:", out1)

# === 5. Zoomed-in window near start (first N seconds) ===
try:
    t0 = df["t"].iloc[0]
    N_sec = min(0.05, (df["t"].iloc[-1] - t0) * 0.1)  # default 50 ms or 10% of duration
    mask = (df["t"] >= t0) & (df["t"] <= t0 + N_sec)
    if mask.sum() < 10:
        # fallback: first 100 samples
        mask = df.index < min(100, len(df))
    fig, axs = plt.subplots(len(channels), 1, figsize=(14, 2.5*len(channels)), sharex=True)
    for ax, ch in zip(axs, channels):
        ax.plot(df.loc[mask, "t"], df.loc[mask, ch], linewidth=0.7)
        ax.set_ylabel(ch)
        ax.grid(True, linewidth=0.3)
    axs[-1].set_xlabel("t (s)")
    fig.suptitle("Waveform zoom (start)")
    out2 = os.path.join(OUT_DIR, "waveform_zoom.png")
    plt.tight_layout(rect=[0,0,1,0.96])
    plt.savefig(out2, dpi=200)
    plt.close(fig)
    print("Saved waveform zoom to:", out2)
except Exception as e:
    print("Could not create zoom plot:", e)

# === 6. Spectrogram of the (assumed) faulted phase ===
# Default to Va if present, otherwise first voltage column
spec_ch = None
for c in ["Va","VA","va","Va "]:
    if c in df.columns:
        spec_ch = c
        break
if spec_ch is None and len(channels_v) > 0:
    spec_ch = channels_v[0]

if spec_ch is not None:
    sig = df[spec_ch].values
    # if t available, compute fs, else assume 10000 Hz for spectrogram default (but warn)
    if "t" in df.columns:
        dt = np.median(np.diff(df["t"].values.astype(float)))
        if dt <= 0:
            fs = None
        else:
            fs = 1.0 / dt
    else:
        fs = None

    if fs is None:
        print("\nCould not determine sampling frequency from 't'. Spectrogram will use default fs=10000 Hz.")
        fs = 10000.0

    f, tt, Sxx = signal.spectrogram(sig, fs=fs, nperseg=256, noverlap=192, scaling="density", mode="magnitude")
    plt.figure(figsize=(12,4))
    plt.pcolormesh(tt, f, np.log1p(Sxx), shading='gouraud')
    plt.ylabel('Frequency [Hz]')
    plt.xlabel('Time [s]')
    plt.title(f"Spectrogram (log1p) - {spec_ch} - fs={fs:.1f} Hz")
    plt.colorbar(label='log magnitude')
    out3 = os.path.join(OUT_DIR, "spectrogram_faulted_phase.png")
    plt.tight_layout()
    plt.savefig(out3, dpi=200)
    plt.close()
    print("Saved spectrogram to:", out3)
else:
    print("\nNo voltage column found for spectrogram.")

# === 7. Quick statistics per channel (mean, std, RMS) for first and last 1s if available ===
print("\nQuick channel statistics (first 1s vs last 1s if t present):")
if "t" in df.columns:
    tmin, tmax = df["t"].min(), df["t"].max()
    dur = tmax - tmin
    window = min(1.0, dur/10.0)  # 1s or 10% of duration
    first_mask = (df["t"] >= tmin) & (df["t"] < tmin + window)
    last_mask = (df["t"] <= tmax) & (df["t"] > tmax - window)
else:
    first_mask = df.index < min(100, len(df))
    last_mask = df.index >= max(0, len(df)-min(100, len(df)))

for ch in channels:
    def stats(mask):
        a = df.loc[mask, ch].dropna().values
        if len(a)==0:
            return (np.nan, np.nan, np.nan)
        return (np.mean(a), np.std(a), np.sqrt(np.mean(a**2)))
    fmean, fstd, frms = stats(first_mask)
    lmean, lstd, lrms = stats(last_mask)
    print(f"{ch}: first mean={fmean:.4g}, std={fstd:.4g}, rms={frms:.4g} | last mean={lmean:.4g}, std={lstd:.4g}, rms={lrms:.4g}")

print("\nStep 1 complete. Please paste the printed console output here and confirm whether the files in the 'eda_outputs' folder were created (waveform_overview.png, waveform_zoom.png, spectrogram_faulted_phase.png).")

Loading: merged_dataset.csv
Loaded dataframe.

=== Basic info ===
Shape: (16004, 8)
Columns: ['t', 'Va', 'Vb', 'Vc', 'Ia', 'Ib', 'Ic', 'Fault']

Dtypes:
 t        float64
Va       float64
Vb       float64
Vc       float64
Ia       float64
Ib       float64
Ic       float64
Fault      int64
dtype: object

First 5 rows:
      t         Va          Vb         Vc          Ia            Ib          Ic  Fault
0.0000 123.673155 -318.714610 195.041455 3827.709068  -9954.681312 6126.972245      0
0.0001 132.928422 -319.851525 186.923103 4117.440004  -9991.463731 5874.023726      0
0.0002 142.052661 -320.672826 178.620166 4403.112401 -10018.387070 5615.274666      0
0.0003 151.036833 -321.177694 170.140861 4684.443276 -10035.424470 5350.981195      0
0.0004 159.872046 -321.365624 161.493578 4961.154162 -10042.558910 5081.404750      0

Last 5 rows:
      t         Va          Vb         Vc          Ia           Ib          Ic  Fault
0.3996  85.518605 -311.035890 225.517284 2633.734420 -9709.76914

In [3]:
# step1_eda_lg_fault_ieee.py
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import signal

# === CONFIG ===
CSV_PATH = "merged_dataset.csv"   # change if your file path differs
OUT_DIR = "eda_outputs"
os.makedirs(OUT_DIR, exist_ok=True)

# === IEEE Publication Plot Settings ===
plt.rcParams.update({
    "font.family": "serif",
    "font.serif": ["Times New Roman", "Times", "DejaVu Serif", "Nimbus Roman No9 L"],
    "font.size": 12,
    "axes.labelsize": 12,
    "axes.titlesize": 14,
    "axes.labelweight": "bold",
    "axes.titleweight": "bold",
    "xtick.labelsize": 12,
    "ytick.labelsize": 12,
    "legend.fontsize": 11,
    "figure.dpi": 120,
    "savefig.dpi": 300,
    "savefig.bbox": "tight",
    "savefig.pad_inches": 0.05,
    "axes.grid": True,
    "grid.alpha": 0.3,
    "grid.linestyle": "--"
})

# === 1. Load ===
print("Loading:", CSV_PATH)
df = pd.read_csv(CSV_PATH)
print("Loaded dataframe.")

# === 2. Basic info ===
print("\n=== Basic info ===")
print("Shape:", df.shape)
print("Columns:", df.columns.tolist())
print("\nDtypes:\n", df.dtypes)
print("\nFirst 5 rows:\n", df.head().to_string(index=False))
print("\nLast 5 rows:\n", df.tail().to_string(index=False))

# Missing values
print("\nMissing values per column:")
print(df.isnull().sum())

# Unique labels for Fault
if "Fault" in df.columns:
    print("\nUnique Fault labels and counts:")
    print(df["Fault"].value_counts(dropna=False))
else:
    print("\nColumn 'Fault' not found in CSV. Please confirm column names.")

# === 3. Sampling interval estimation ===
if "t" in df.columns:
    t = df["t"].values.astype(float)
    dt = np.diff(t)
    dt = dt[np.isfinite(dt) & (dt > 0)]
    if len(dt) == 0:
        print("\nNo positive time differences found in 't' to estimate sampling interval.")
    else:
        print("\nSampling interval (seconds):")
        print(" count:", len(dt))
        print(" mean: {:.6f}".format(np.mean(dt)))
        print(" median: {:.6f}".format(np.median(dt)))
        print(" std: {:.6f}".format(np.std(dt)))
        fs_median = 1.0 / np.median(dt)
        print(" approx sampling frequency (Hz) from median dt: {:.3f} Hz".format(fs_median))
else:
    print("\nColumn 't' not found in CSV. Please confirm column names.")

# === 4. Waveform overview ===
channels_v = [c for c in df.columns if c.lower().startswith("v")]
channels_i = [c for c in df.columns if c.lower().startswith("i")]
channels = channels_v + channels_i

if len(channels) == 0:
    print("\nNo Va/Vb/Vc/Ia/Ib/Ic columns detected by prefix. Detected columns:", df.columns.tolist())
else:
    fig, axs = plt.subplots(len(channels), 1, figsize=(10, 2.2 * len(channels)), sharex=True)
    for ax, ch in zip(axs, channels):
        ax.plot(df["t"], df[ch], linewidth=0.8)
        if ch.lower().startswith("v"):
            ax.set_ylabel("V (Volts)", fontweight='bold')
        elif ch.lower().startswith("i"):
            ax.set_ylabel("I (Amperes)", fontweight='bold')
        ax.set_title(ch, fontweight='bold', loc='left', fontsize=12)
    axs[-1].set_xlabel("Time (s)", fontweight='bold')
    fig.suptitle("Waveform Overview", fontweight='bold')
    out1 = os.path.join(OUT_DIR, "waveform_overview.png")
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.savefig(out1, format="png")
    plt.savefig(out1.replace(".png", ".pdf"), format="pdf")
    plt.close(fig)
    print("\nSaved waveform overview to:", out1)


# === 5. Zoomed waveform ===
try:
    t0 = df["t"].iloc[0]
    N_sec = min(0.05, (df["t"].iloc[-1] - t0) * 0.1)
    mask = (df["t"] >= t0) & (df["t"] <= t0 + N_sec)
    if mask.sum() < 10:
        mask = df.index < min(100, len(df))
    fig, axs = plt.subplots(len(channels), 1, figsize=(10, 2.2 * len(channels)), sharex=True)
    for ax, ch in zip(axs, channels):
        ax.plot(df.loc[mask, "t"], df.loc[mask, ch], linewidth=0.9)
        if ch.lower().startswith("v"):
            ax.set_ylabel("V (Volts)", fontweight='bold')
        elif ch.lower().startswith("i"):
            ax.set_ylabel("I (Amperes)", fontweight='bold')
        ax.set_title(ch, fontweight='bold', loc='left', fontsize=12)
    axs[-1].set_xlabel("Time (s)", fontweight='bold')
    fig.suptitle("Waveform Zoom (Initial 50 ms)", fontweight='bold')
    out2 = os.path.join(OUT_DIR, "waveform_zoom.png")
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.savefig(out2, format="png")
    plt.savefig(out2.replace(".png", ".pdf"), format="pdf")
    plt.close(fig)
    print("Saved waveform zoom to:", out2)
except Exception as e:
    print("Could not create zoom plot:", e)

# === 6. Spectrogram of faulted phase ===
spec_ch = None
for c in ["Va", "VA", "va", "Va "]:
    if c in df.columns:
        spec_ch = c
        break
if spec_ch is None and len(channels_v) > 0:
    spec_ch = channels_v[0]

if spec_ch is not None:
    sig = df[spec_ch].values
    if "t" in df.columns:
        dt = np.median(np.diff(df["t"].values.astype(float)))
        fs = 1.0 / dt if dt > 0 else None
    else:
        fs = None
    if fs is None:
        print("\nCould not determine sampling frequency from 't'. Using fs=10000 Hz.")
        fs = 10000.0

    f, tt, Sxx = signal.spectrogram(sig, fs=fs, nperseg=256, noverlap=192,
                                    scaling="density", mode="magnitude")
    plt.figure(figsize=(10, 4))
    plt.pcolormesh(tt, f, np.log1p(Sxx), shading='gouraud')
    plt.ylabel('Frequency (Hz)', fontweight='bold')
    plt.xlabel('Time (s)', fontweight='bold')
    plt.title(f"Spectrogram (log scale) - {spec_ch}", fontweight='bold')
    cbar = plt.colorbar()
    cbar.set_label("log Magnitude", fontweight='bold')
    out3 = os.path.join(OUT_DIR, "spectrogram_faulted_phase.png")
    plt.tight_layout()
    plt.savefig(out3, format="png")
    plt.savefig(out3.replace(".png", ".pdf"), format="pdf")
    plt.close()
    print("Saved spectrogram to:", out3)
else:
    print("\nNo voltage column found for spectrogram.")

# === 7. Quick stats ===
print("\nQuick channel statistics (first 1s vs last 1s if t present):")
if "t" in df.columns:
    tmin, tmax = df["t"].min(), df["t"].max()
    dur = tmax - tmin
    window = min(1.0, dur / 10.0)
    first_mask = (df["t"] >= tmin) & (df["t"] < tmin + window)
    last_mask = (df["t"] <= tmax) & (df["t"] > tmax - window)
else:
    first_mask = df.index < min(100, len(df))
    last_mask = df.index >= max(0, len(df) - min(100, len(df)))

for ch in channels:
    def stats(mask):
        a = df.loc[mask, ch].dropna().values
        if len(a) == 0:
            return (np.nan, np.nan, np.nan)
        return (np.mean(a), np.std(a), np.sqrt(np.mean(a**2)))

    fmean, fstd, frms = stats(first_mask)
    lmean, lstd, lrms = stats(last_mask)
    print(f"{ch}: first mean={fmean:.4g}, std={fstd:.4g}, rms={frms:.4g} | "
          f"last mean={lmean:.4g}, std={lstd:.4g}, rms={lrms:.4g}")

print("\nStep 1 complete. Check 'eda_outputs/' for IEEE-quality figures.")

Loading: merged_dataset.csv
Loaded dataframe.

=== Basic info ===
Shape: (16004, 8)
Columns: ['t', 'Va', 'Vb', 'Vc', 'Ia', 'Ib', 'Ic', 'Fault']

Dtypes:
 t        float64
Va       float64
Vb       float64
Vc       float64
Ia       float64
Ib       float64
Ic       float64
Fault      int64
dtype: object

First 5 rows:
      t         Va          Vb         Vc          Ia            Ib          Ic  Fault
0.0000 123.673155 -318.714610 195.041455 3827.709068  -9954.681312 6126.972245      0
0.0001 132.928422 -319.851525 186.923103 4117.440004  -9991.463731 5874.023726      0
0.0002 142.052661 -320.672826 178.620166 4403.112401 -10018.387070 5615.274666      0
0.0003 151.036833 -321.177694 170.140861 4684.443276 -10035.424470 5350.981195      0
0.0004 159.872046 -321.365624 161.493578 4961.154162 -10042.558910 5081.404750      0

Last 5 rows:
      t         Va          Vb         Vc          Ia           Ib          Ic  Fault
0.3996  85.518605 -311.035890 225.517284 2633.734420 -9709.76914

In [9]:
# step2b_detect_onsets_robust.py
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import medfilt, find_peaks

# === CONFIG ===
CSV_PATH = "merged_dataset.csv"
OUT_DIR = "windows_outputs_robust"
PLOT_DIR = os.path.join(OUT_DIR, "plots")
NPZ_DIR = os.path.join(OUT_DIR, "npz")
META_CSV = os.path.join(OUT_DIR, "windows_metadata.csv")
os.makedirs(OUT_DIR, exist_ok=True)
os.makedirs(PLOT_DIR, exist_ok=True)
os.makedirs(NPZ_DIR, exist_ok=True)

PRE_SEC = 0.02    # seconds before onset
POST_SEC = 0.2    # seconds after onset
MAX_EVENTS = 20   # maximum windows to save (top-K if many candidates)
MIN_EVENT_SEP_SEC = 0.05  # min separation between events when peak-picking

# detection hyperparameters (you can increase sensitivity by lowering these)
SMOOTH_KERNEL = 51   # median filter kernel (odd)
ENERGY_STD_FACTOR = 4.0  # threshold = median + factor*std (lower -> more sensitive)
DERIV_PROMINENCE = None  # None -> compute from derivative statistics
VAR_WIN_MS = 5.0  # window width (ms) for short-time variance
TOPK_FALLBACK = True  # if no peaks found, choose top-K largest derivative magnitudes

print("Robust onset detection: loading", CSV_PATH)
df = pd.read_csv(CSV_PATH)
print("Shape:", df.shape)
cols = df.columns.tolist()
tcol = next((c for c in cols if c.lower()=="t"), None)
if tcol is None:
    raise RuntimeError("Timestamp column 't' not found.")
t = df[tcol].values.astype(float)
dt_arr = np.diff(t)
dt_good = dt_arr[np.isfinite(dt_arr) & (dt_arr>0)]
if len(dt_good)==0:
    raise RuntimeError("Could not estimate positive time differences in 't'.")
dt = np.median(dt_good)
fs = 1.0/dt
print(f"Estimated dt={dt:.6e} s -> fs={fs:.3f} Hz")

voltage_cols = [c for c in cols if c.lower().startswith("v")]
current_cols = [c for c in cols if c.lower().startswith("i")]
print("Voltage cols:", voltage_cols)
print("Current cols:", current_cols)

V = df[voltage_cols].values if len(voltage_cols)>0 else None
I = df[current_cols].values if len(current_cols)>0 else None

# --- Build signals used for detection ---
# 1) energy proxy: sum abs voltages
energy = np.sum(np.abs(V), axis=1) if V is not None else np.sum(np.abs(I), axis=1)
k = SMOOTH_KERNEL if SMOOTH_KERNEL%2==1 else SMOOTH_KERNEL+1
energy_smooth = medfilt(energy, k)
med_e, std_e = np.median(energy_smooth), np.std(energy_smooth)
energy_thresh = med_e + ENERGY_STD_FACTOR * std_e

# 2) derivative of sum of voltages (abs derivative)
sig_for_deriv = np.sum(V, axis=1) if V is not None else energy
deriv = np.abs(np.concatenate(([0], np.diff(sig_for_deriv)))) / dt
# smooth derivative with small median
deriv_smooth = medfilt(deriv, 5)

# 3) short-time variance (sliding window)
win_len = max(1, int(round((VAR_WIN_MS/1000.0) * fs)))
if win_len % 2 == 0: win_len += 1
pad = win_len // 2
arr = sig_for_deriv
sq = arr**2
cumsum = np.concatenate(([0], np.cumsum(sq)))
# windowed mean of square -> variance proxy
sts_var = (cumsum[win_len:] - cumsum[:-win_len]) / float(win_len)
sts_var = np.concatenate((np.full(pad, sts_var[0]), sts_var, np.full(pad, sts_var[-1])))

print(f"Computed energy (median={med_e:.4g}, std={std_e:.4g}, thresh={energy_thresh:.4g}), derivative and short-time var (win {win_len} samples).")

# --- Peak picking on each detector ---
min_dist_samples = int(round(MIN_EVENT_SEP_SEC * fs))
candidates = set()

# energy threshold rising edges
above = energy_smooth > energy_thresh
rising = np.nonzero(np.logical_and(above, ~np.concatenate(([False], above[:-1]))))[0]
print("Energy-threshold rising-edge candidates:", len(rising))
for r in rising: candidates.add(int(r))

# derivative peaks (prominence-based)
if DERIV_PROMINENCE is None:
    prom = np.median(deriv_smooth) + 3.0*np.std(deriv_smooth)
else:
    prom = DERIV_PROMINENCE
peaks_deriv, prop = find_peaks(deriv_smooth, prominence=prom, distance=min_dist_samples)
print("Derivative-based peaks found:", len(peaks_deriv), " (prominence used =", prom, ")")
for p in peaks_deriv: candidates.add(int(p))

# short-time variance peaks
prom_var = np.median(sts_var) + 3.0*np.std(sts_var)
peaks_var, _ = find_peaks(sts_var, prominence=prom_var, distance=min_dist_samples)
print("Short-time-variance peaks found:", len(peaks_var), " (prom_var =", prom_var, ")")
for p in peaks_var: candidates.add(int(p))

candidates = np.array(sorted(list(candidates)))
print("Total unique candidates after combining detectors:", len(candidates))

# If no candidates, fallback: choose top-K largest derivative magnitudes
if len(candidates) == 0 and TOPK_FALLBACK:
    K = MAX_EVENTS
    idx_sorted = np.argsort(deriv_smooth)[-K:]
    candidates = np.sort(idx_sorted)
    print(f"No peaks detected; fallback selected top-{len(candidates)} derivative indices as candidates.")

# Merge candidates to enforce min separation: pick within cluster the index with max energy jump
if len(candidates) > 0:
    merged = []
    curr = candidates[0]
    cluster = [curr]
    for idx in candidates[1:]:
        if idx - cluster[-1] <= min_dist_samples:
            cluster.append(idx)
        else:
            # pick best representative (max derivative * energy jump)
            cluster = np.array(cluster)
            scores = deriv_smooth[cluster] * (energy_smooth[cluster] - med_e)
            best = cluster[np.argmax(scores)]
            merged.append(int(best))
            cluster = [idx]
    # last cluster
    cluster = np.array(cluster)
    scores = deriv_smooth[cluster] * (energy_smooth[cluster] - med_e)
    best = cluster[np.argmax(scores)]
    merged.append(int(best))
    candidates = np.array(merged)
    print("Candidates after merging by min separation:", len(candidates))

# Score candidates and take top-K by score
scores = (deriv_smooth[candidates] * (energy_smooth[candidates] - med_e))
order = np.argsort(scores)[::-1]
selected = candidates[order][:MAX_EVENTS]
selected = np.sort(selected)
print("Selected events (indices):", selected.tolist())
print("Corresponding times (s):", [float(t[int(i)]) for i in selected])

# Filter out those too close to edges for window extraction
pre_samples = int(round(PRE_SEC * fs))
post_samples = int(round(POST_SEC * fs))
valid_selected = [int(i) for i in selected if i - pre_samples >= 0 and i + post_samples < len(df)]
print("Valid selected events (full window fits):", len(valid_selected))

# Extract windows and save
rows = []
for j, onset_idx in enumerate(valid_selected):
    start_idx = onset_idx - pre_samples
    end_idx = onset_idx + post_samples
    t_window = t[start_idx:end_idx+1]
    V_window = V[start_idx:end_idx+1] if V is not None else None
    I_window = I[start_idx:end_idx+1] if I is not None else None

    # heuristic phase detection (relative RMS drop)
    phase_detect = None
    phase_scores = {}
    if V_window is not None:
        pre_seg = V_window[:pre_samples]
        post_seg = V_window[pre_samples:pre_samples + max(1, int(round(0.05*fs)))]
        for pi, ch in enumerate(voltage_cols):
            pre_rms = np.sqrt(np.mean(pre_seg[:,pi]**2)) if len(pre_seg)>0 else 0.0
            post_rms = np.sqrt(np.mean(post_seg[:,pi]**2)) if len(post_seg)>0 else 0.0
            score = (pre_rms - post_rms) / (pre_rms + 1e-12)
            phase_scores[ch] = float(score)
        phase_detect = max(phase_scores.items(), key=lambda x: x[1])[0]

    # stats
    peak_curr = float(np.max(np.abs(I_window))) if I_window is not None else np.nan
    pre_rms_all = float(np.sqrt(np.mean(V_window[:pre_samples,:]**2))) if V_window is not None else np.nan
    post_rms_all = float(np.sqrt(np.mean(V_window[pre_samples:pre_samples+max(1,int(round(0.05*fs))),:]**2))) if V_window is not None else np.nan

    fname = f"window_{j:03d}_onsetidx{onset_idx}.npz"
    fpath = os.path.join(NPZ_DIR, fname)
    np.savez_compressed(fpath, t=t_window, V=V_window, I=I_window, onset_idx=int(onset_idx),
                        start_idx=int(start_idx), end_idx=int(end_idx))
    rows.append({
        "window_file": fname,
        "onset_idx": int(onset_idx),
        "onset_time": float(t[onset_idx]),
        "start_idx": int(start_idx),
        "start_time": float(t[start_idx]),
        "end_idx": int(end_idx),
        "end_time": float(t[end_idx]),
        "detected_phase": phase_detect,
        "phase_scores": str(phase_scores),
        "deriv_at_onset": float(deriv_smooth[onset_idx]),
        "energy_at_onset": float(energy_smooth[onset_idx]),
        "score": float(scores[np.where(candidates==onset_idx)[0][0]]) if onset_idx in candidates else float(deriv_smooth[onset_idx]),
        "peak_current": peak_curr,
        "pre_rms_all": pre_rms_all,
        "post_rms_all": post_rms_all,
        "n_samples": int(end_idx - start_idx + 1)
    })

    # save per-window plot
    fig, axs = plt.subplots(2,1, figsize=(10,5), sharex=True)
    if V_window is not None:
        for pi, ch in enumerate(voltage_cols):
            axs[0].plot(t_window - t_window[0], V_window[:,pi], label=ch, linewidth=0.8)
        axs[0].legend(fontsize='small')
    if I_window is not None:
        for pi, ch in enumerate(current_cols):
            axs[1].plot(t_window - t_window[0], I_window[:,pi], label=ch, linewidth=0.8)
        axs[1].legend(fontsize='small')
    axs[0].axvline(x=pre_samples*dt, color='k', linestyle='--')
    axs[1].axvline(x=pre_samples*dt, color='k', linestyle='--')
    axs[1].set_xlabel("time (s) rel window start")
    axs[0].set_title(f"Window {j:03d} onset@{t[onset_idx]:.6f}s detected_phase={phase_detect}")
    plt.tight_layout()
    plt.savefig(os.path.join(PLOT_DIR, f"window_{j:03d}.png"), dpi=150)
    plt.close(fig)

# Save metadata
meta_df = pd.DataFrame(rows)
meta_df.to_csv(META_CSV, index=False)
print("\nSaved metadata to:", META_CSV)
print("Saved NPZ windows to:", NPZ_DIR)
print("Saved per-window plots to:", PLOT_DIR)
print("\nSummary:")
print(" Total candidates considered:", len(candidates))
print(" Total windows saved:", len(meta_df))
if len(meta_df)>0:
    print("\nFirst rows of metadata:")
    with pd.option_context('display.max_colwidth', 200):
        print(meta_df.head(8).to_string(index=False))

# Also save an overview plot with detected events marked
fig, ax = plt.subplots(1,1, figsize=(14,4))
ax.plot(t, energy_smooth, linewidth=0.6, label='energy_smooth (sum|V|)')
ax.axhline(energy_thresh, color='r', linestyle='--', label='energy_thresh')
for onset_idx in selected:
    ax.axvline(t[onset_idx], color='k', alpha=0.6)
ax.set_xlabel("t (s)")
ax.set_ylabel("energy proxy")
ax.set_title("Overview: detected events marked")
ax.legend(fontsize='small')
plt.tight_layout()
overview_png = os.path.join(OUT_DIR, "overview_detected_events.png")
plt.savefig(overview_png, dpi=150)
plt.close(fig)
print("Saved overview plot:", overview_png)

print("\nStep complete. Please paste the console output here and confirm:")
print(" - How many windows were saved?")
print(" - Are NPZ files present in:", NPZ_DIR)
print(" - Paste first <=8 rows from the saved metadata CSV if present.")

Robust onset detection: loading merged_dataset.csv
Shape: (16004, 8)
Estimated dt=1.000000e-04 s -> fs=10000.000 Hz
Voltage cols: ['Va', 'Vb', 'Vc']
Current cols: ['Ia', 'Ib', 'Ic']
Computed energy (median=613.9, std=172.7, thresh=1305), derivative and short-time var (win 51 samples).
Energy-threshold rising-edge candidates: 0
Derivative-based peaks found: 4  (prominence used = 110827.2833850379 )
Short-time-variance peaks found: 4  (prom_var = 116794.91909200564 )
Total unique candidates after combining detectors: 8
Candidates after merging by min separation: 1
Selected events (indices): [2141]
Corresponding times (s): [0.2141]
Valid selected events (full window fits): 1

Saved metadata to: windows_outputs_robust\windows_metadata.csv
Saved NPZ windows to: windows_outputs_robust\npz
Saved per-window plots to: windows_outputs_robust\plots

Summary:
 Total candidates considered: 1
 Total windows saved: 1

First rows of metadata:
                window_file  onset_idx  onset_time  start_i

In [10]:
# step2b_detect_onsets_robust_ieee.py
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import medfilt, find_peaks

# === CONFIG ===
CSV_PATH = "merged_dataset.csv"
OUT_DIR = "windows_outputs_robust"
PLOT_DIR = os.path.join(OUT_DIR, "plots")
NPZ_DIR = os.path.join(OUT_DIR, "npz")
META_CSV = os.path.join(OUT_DIR, "windows_metadata.csv")
os.makedirs(OUT_DIR, exist_ok=True)
os.makedirs(PLOT_DIR, exist_ok=True)
os.makedirs(NPZ_DIR, exist_ok=True)

# === IEEE Publication Plot Settings ===
plt.rcParams.update({
    "font.family": "serif",
    "font.serif": ["Times New Roman", "Times", "DejaVu Serif", "Nimbus Roman No9 L"],
    "font.size": 12,
    "axes.labelsize": 12,
    "axes.titlesize": 14,
    "axes.labelweight": "bold",
    "axes.titleweight": "bold",
    "xtick.labelsize": 12,
    "ytick.labelsize": 12,
    "legend.fontsize": 11,
    "figure.dpi": 120,
    "savefig.dpi": 300,
    "savefig.bbox": "tight",
    "savefig.pad_inches": 0.05,
    "axes.grid": True,
    "grid.alpha": 0.3,
    "grid.linestyle": "--"
})

# === PARAMETERS ===
PRE_SEC = 0.02
POST_SEC = 0.2
MAX_EVENTS = 20
MIN_EVENT_SEP_SEC = 0.05
SMOOTH_KERNEL = 51
ENERGY_STD_FACTOR = 4.0
DERIV_PROMINENCE = None
VAR_WIN_MS = 5.0
TOPK_FALLBACK = True

print("Robust onset detection: loading", CSV_PATH)
df = pd.read_csv(CSV_PATH)
print("Shape:", df.shape)

cols = df.columns.tolist()
tcol = next((c for c in cols if c.lower() == "t"), None)
if tcol is None:
    raise RuntimeError("Timestamp column 't' not found.")
t = df[tcol].values.astype(float)
dt_arr = np.diff(t)
dt_good = dt_arr[np.isfinite(dt_arr) & (dt_arr > 0)]
if len(dt_good) == 0:
    raise RuntimeError("Could not estimate positive time differences in 't'.")
dt = np.median(dt_good)
fs = 1.0 / dt
print(f"Estimated dt={dt:.6e} s -> fs={fs:.3f} Hz")

voltage_cols = [c for c in cols if c.lower().startswith("v")]
current_cols = [c for c in cols if c.lower().startswith("i")]
print("Voltage cols:", voltage_cols)
print("Current cols:", current_cols)

V = df[voltage_cols].values if len(voltage_cols) > 0 else None
I = df[current_cols].values if len(current_cols) > 0 else None

# --- Energy and features ---
energy = np.sum(np.abs(V), axis=1) if V is not None else np.sum(np.abs(I), axis=1)
k = SMOOTH_KERNEL if SMOOTH_KERNEL % 2 == 1 else SMOOTH_KERNEL + 1
energy_smooth = medfilt(energy, k)
med_e, std_e = np.median(energy_smooth), np.std(energy_smooth)
energy_thresh = med_e + ENERGY_STD_FACTOR * std_e
sig_for_deriv = np.sum(V, axis=1) if V is not None else energy
deriv = np.abs(np.concatenate(([0], np.diff(sig_for_deriv)))) / dt
deriv_smooth = medfilt(deriv, 5)

# short-time variance
win_len = max(1, int(round((VAR_WIN_MS / 1000.0) * fs)))
if win_len % 2 == 0: win_len += 1
pad = win_len // 2
arr = sig_for_deriv
sq = arr ** 2
cumsum = np.concatenate(([0], np.cumsum(sq)))
sts_var = (cumsum[win_len:] - cumsum[:-win_len]) / float(win_len)
sts_var = np.concatenate((np.full(pad, sts_var[0]), sts_var, np.full(pad, sts_var[-1])))

print(f"Computed energy (median={med_e:.4g}, std={std_e:.4g}, thresh={energy_thresh:.4g}).")

# --- Peak detection ---
min_dist_samples = int(round(MIN_EVENT_SEP_SEC * fs))
candidates = set()

above = energy_smooth > energy_thresh
rising = np.nonzero(np.logical_and(above, ~np.concatenate(([False], above[:-1]))))[0]
for r in rising: candidates.add(int(r))

if DERIV_PROMINENCE is None:
    prom = np.median(deriv_smooth) + 3.0 * np.std(deriv_smooth)
else:
    prom = DERIV_PROMINENCE
peaks_deriv, _ = find_peaks(deriv_smooth, prominence=prom, distance=min_dist_samples)
for p in peaks_deriv: candidates.add(int(p))

prom_var = np.median(sts_var) + 3.0 * np.std(sts_var)
peaks_var, _ = find_peaks(sts_var, prominence=prom_var, distance=min_dist_samples)
for p in peaks_var: candidates.add(int(p))

candidates = np.array(sorted(list(candidates)))
if len(candidates) == 0 and TOPK_FALLBACK:
    K = MAX_EVENTS
    idx_sorted = np.argsort(deriv_smooth)[-K:]
    candidates = np.sort(idx_sorted)

# Merge nearby events
if len(candidates) > 0:
    merged = []
    curr = candidates[0]
    cluster = [curr]
    for idx in candidates[1:]:
        if idx - cluster[-1] <= min_dist_samples:
            cluster.append(idx)
        else:
            cluster = np.array(cluster)
            scores = deriv_smooth[cluster] * (energy_smooth[cluster] - med_e)
            best = cluster[np.argmax(scores)]
            merged.append(int(best))
            cluster = [idx]
    cluster = np.array(cluster)
    scores = deriv_smooth[cluster] * (energy_smooth[cluster] - med_e)
    best = cluster[np.argmax(scores)]
    merged.append(int(best))
    candidates = np.array(merged)

# Select top events
scores = (deriv_smooth[candidates] * (energy_smooth[candidates] - med_e))
order = np.argsort(scores)[::-1]
selected = candidates[order][:MAX_EVENTS]
selected = np.sort(selected)

pre_samples = int(round(PRE_SEC * fs))
post_samples = int(round(POST_SEC * fs))
valid_selected = [int(i) for i in selected if i - pre_samples >= 0 and i + post_samples < len(df)]

rows = []
for j, onset_idx in enumerate(valid_selected):
    start_idx = onset_idx - pre_samples
    end_idx = onset_idx + post_samples
    t_window = t[start_idx:end_idx + 1]
    V_window = V[start_idx:end_idx + 1] if V is not None else None
    I_window = I[start_idx:end_idx + 1] if I is not None else None

    # phase detection
    phase_detect = None
    phase_scores = {}
    if V_window is not None:
        pre_seg = V_window[:pre_samples]
        post_seg = V_window[pre_samples:pre_samples + max(1, int(round(0.05 * fs)))]
        for pi, ch in enumerate(voltage_cols):
            pre_rms = np.sqrt(np.mean(pre_seg[:, pi] ** 2)) if len(pre_seg) > 0 else 0.0
            post_rms = np.sqrt(np.mean(post_seg[:, pi] ** 2)) if len(post_seg) > 0 else 0.0
            score = (pre_rms - post_rms) / (pre_rms + 1e-12)
            phase_scores[ch] = float(score)
        phase_detect = max(phase_scores.items(), key=lambda x: x[1])[0]

    # per-window plot === IEEE Style ===
    fig, axs = plt.subplots(2, 1, figsize=(10, 5), sharex=True)
    if V_window is not None:
        for pi, ch in enumerate(voltage_cols):
            axs[0].plot(t_window - t_window[0], V_window[:, pi], label=ch, linewidth=0.8)
        axs[0].set_ylabel("V (Volts)", fontweight='bold')
        axs[0].legend(loc='upper right')
    if I_window is not None:
        for pi, ch in enumerate(current_cols):
            axs[1].plot(t_window - t_window[0], I_window[:, pi], label=ch, linewidth=0.8)
        axs[1].set_ylabel("I (Amperes)", fontweight='bold')
        axs[1].legend(loc='upper right')
    axs[1].set_xlabel("Time (s)", fontweight='bold')
    axs[0].axvline(x=pre_samples * dt, color='k', linestyle='--')
    axs[1].axvline(x=pre_samples * dt, color='k', linestyle='--')
    axs[0].set_title(f"Window {j:03d} | Onset @ {t[onset_idx]:.6f}s | Phase = {phase_detect}", fontweight='bold')
    plt.tight_layout()
    figpath = os.path.join(PLOT_DIR, f"window_{j:03d}.png")
    plt.savefig(figpath, format="png")
    plt.savefig(figpath.replace(".png", ".pdf"), format="pdf")
    plt.close(fig)

# --- Overview Plot (IEEE Style) ---
fig, ax = plt.subplots(1, 1, figsize=(12, 4))
ax.plot(t, energy_smooth, linewidth=0.8, label='Energy (sum|V|)')
ax.axhline(energy_thresh, color='r', linestyle='--', label='Threshold')
for onset_idx in selected:
    ax.axvline(t[onset_idx], color='k', linestyle='--', alpha=0.6)
ax.set_xlabel("Time (s)", fontweight='bold')
ax.set_ylabel("Energy Proxy (a.u.)", fontweight='bold')
ax.set_title("Detected Event Onsets", fontweight='bold')
ax.legend(loc='upper right')
plt.tight_layout()
overview_png = os.path.join(OUT_DIR, "overview_detected_events.png")
plt.savefig(overview_png, format="png")
plt.savefig(overview_png.replace(".png", ".pdf"), format="pdf")
plt.close(fig)
print("Saved overview plot:", overview_png)

Robust onset detection: loading merged_dataset.csv
Shape: (16004, 8)
Estimated dt=1.000000e-04 s -> fs=10000.000 Hz
Voltage cols: ['Va', 'Vb', 'Vc']
Current cols: ['Ia', 'Ib', 'Ic']
Computed energy (median=613.9, std=172.7, thresh=1305).
Saved overview plot: windows_outputs_robust\overview_detected_events.png


In [11]:
# === Combined Summary Plot (all windows stacked vertically) ===
if len(valid_selected) > 0:
    print("Creating combined summary plot for all detected windows...")

    # Determine time axis length (all windows same size)
    window_len = pre_samples + post_samples + 1
    time_axis = np.linspace(-PRE_SEC, POST_SEC, window_len)

    # Prepare figure
    n_windows = len(valid_selected)
    fig, axs = plt.subplots(n_windows, 1, figsize=(10, 2.2 * n_windows), sharex=True)

    if n_windows == 1:
        axs = [axs]

    for j, (ax, onset_idx) in enumerate(zip(axs, valid_selected)):
        start_idx = onset_idx - pre_samples
        end_idx = onset_idx + post_samples
        V_window = V[start_idx:end_idx + 1] if V is not None else None
        I_window = I[start_idx:end_idx + 1] if I is not None else None

        # Voltage or Current selection for clarity
        if V_window is not None:
            for pi, ch in enumerate(voltage_cols):
                ax.plot(time_axis, V_window[:, pi], label=ch, linewidth=0.8)
            ax.set_ylabel("V (Volts)", fontweight="bold")
        elif I_window is not None:
            for pi, ch in enumerate(current_cols):
                ax.plot(time_axis, I_window[:, pi], label=ch, linewidth=0.8)
            ax.set_ylabel("I (Amperes)", fontweight="bold")

        # Mark the onset
        ax.axvline(0.0, color="k", linestyle="--", linewidth=1)
        ax.grid(True, linewidth=0.3)
        ax.set_title(f"Event {j+1}: Onset @ {t[onset_idx]:.6f} s", fontweight="bold", loc="left", fontsize=12)

        if j == 0:
            ax.legend(loc="upper right")

    axs[-1].set_xlabel("Time (s)", fontweight="bold")

    fig.suptitle("Combined Summary of Detected Event Windows", fontweight="bold", fontsize=14)
    plt.tight_layout(rect=[0, 0, 1, 0.97])

    summary_path = os.path.join(PLOT_DIR, "combined_summary_windows.png")
    plt.savefig(summary_path, format="png")
    plt.savefig(summary_path.replace(".png", ".pdf"), format="pdf")
    plt.close(fig)

    print("Saved combined summary figure:", summary_path)
else:
    print("No valid detected windows available for summary plot.")

Creating combined summary plot for all detected windows...
Saved combined summary figure: windows_outputs_robust\plots\combined_summary_windows.png


In [16]:
# diagnose_windows_dataset.py
import os, glob, numpy as np, pandas as pd

paths = [
    "windows_outputs_robust/npz",
    "windows_outputs/npz",
    "windows_outputs_robust",
    "windows_outputs"
]
npz_folder = None
for p in paths:
    if os.path.isdir(p):
        files = glob.glob(os.path.join(p, "*.npz"))
        if len(files) > 0:
            npz_folder = p
            break
if npz_folder is None:
    print("❌ No .npz found — please rerun window extraction.")
else:
    print("✅ NPZ folder:", npz_folder)
    files = sorted(glob.glob(os.path.join(npz_folder, "*.npz")))
    print("Total NPZ files:", len(files))

meta_candidates = [
    os.path.join(os.path.dirname(npz_folder), "windows_metadata.csv"),
    os.path.join(".", "windows_metadata.csv"),
]
meta = None
for m in meta_candidates:
    if os.path.isfile(m):
        try:
            meta = pd.read_csv(m)
            print("✅ Loaded metadata from:", m)
            break
        except Exception as e:
            print("Failed to load metadata", m, "->", e)
            meta = None

if meta is not None:
    print("\nMetadata columns:", meta.columns.tolist())
    print("Total metadata rows:", len(meta))
    if "detected_phase" in meta.columns:
        print("\nDetected phase distribution:")
        print(meta["detected_phase"].value_counts(dropna=False))
    else:
        print("⚠️ 'detected_phase' column not found.")
else:
    print("⚠️ No metadata CSV found near the npz folder.")

# Try to load one example .npz
if npz_folder is not None and len(files)>0:
    z = np.load(files[0], allow_pickle=True)
    print("\nExample file:", files[0])
    for k in z.keys():
        arr = z[k]
        if hasattr(arr, "shape"):
            print(f"  {k}: shape={arr.shape}, dtype={arr.dtype}")
        else:
            print(f"  {k}: {arr}")

✅ NPZ folder: windows_outputs_robust/npz
Total NPZ files: 1
✅ Loaded metadata from: windows_outputs_robust\windows_metadata.csv

Metadata columns: ['window_file', 'onset_idx', 'onset_time', 'start_idx', 'start_time', 'end_idx', 'end_time', 'detected_phase', 'phase_scores', 'deriv_at_onset', 'energy_at_onset', 'score', 'peak_current', 'pre_rms_all', 'post_rms_all', 'n_samples']
Total metadata rows: 1

Detected phase distribution:
detected_phase
Va    1
Name: count, dtype: int64

Example file: windows_outputs_robust/npz\window_000_onsetidx2141.npz
  t: shape=(2201,), dtype=float64
  V: shape=(2201, 3), dtype=float64
  I: shape=(2201, 3), dtype=float64
  onset_idx: shape=(), dtype=int32
  start_idx: shape=(), dtype=int32
  end_idx: shape=(), dtype=int32


In [17]:
# step3_aug_and_train_ml.py
import os, glob, math, random, warnings
warnings.filterwarnings("ignore")
import numpy as np, pandas as pd
from scipy import stats, signal
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import joblib

# Optional models
try:
    import xgboost as xgb
except Exception:
    xgb = None
try:
    import lightgbm as lgb
except Exception:
    lgb = None
try:
    from catboost import CatBoostClassifier
except Exception:
    CatBoostClassifier = None

# === CONFIG ===
NPZ_FOLDER = "windows_outputs_robust/npz"
OUT_DIR = "ml_aug_out"
os.makedirs(OUT_DIR, exist_ok=True)
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)
random.seed(RANDOM_STATE)

N_AUG = 400           # number of augmented samples to create (choose 200-1000)
KEEP_ORIGINAL = True  # include the original window as one sample
TEST_SIZE = 0.15
VAL_SIZE = 0.15
CV_FOLDS = 5

# augmentation hyperparams
NOISE_STD_REL = 0.02   # noise std as fraction of signal RMS
SCALE_MIN, SCALE_MAX = 0.8, 1.2
TIME_SHIFT_MAX_SAMPLES = 50  # shift left/right by up to this many samples (zero-pad)
TIME_STRETCH_MIN, TIME_STRETCH_MAX = 0.9, 1.1  # resample factor
CROP_MIN_FRAC = 0.8   # random crop keeps between 80% and 100% of length, then pad back

# feature helpers (same as before)
from scipy.signal import welch

def time_features(ch):
    f = {}
    f['mean'] = np.mean(ch)
    f['std'] = np.std(ch)
    f['rms'] = np.sqrt(np.mean(ch**2))
    f['ptp'] = np.ptp(ch)
    f['abs_mean'] = np.mean(np.abs(ch))
    f['skew'] = float(stats.skew(ch))
    f['kurtosis'] = float(stats.kurtosis(ch))
    f['median'] = np.median(ch)
    f['max'] = np.max(ch)
    f['min'] = np.min(ch)
    f['energy'] = np.sum(ch**2)
    return f

def spectral_features(ch, fs):
    try:
        f, Pxx = welch(ch, fs=fs, nperseg=min(256, len(ch)))
        sc = np.sum(f * Pxx) / (np.sum(Pxx) + 1e-12)
        dom = f[np.argmax(Pxx)]
        nyq = fs / 2.0
        low = np.sum(Pxx[(f >= 0) & (f < 0.1*nyq)])
        mid = np.sum(Pxx[(f >= 0.1*nyq) & (f < 0.4*nyq)])
        high = np.sum(Pxx[(f >= 0.4*nyq)])
        return {'spec_centroid': sc, 'spec_domfreq': dom, 'spec_low_energy': low, 'spec_mid_energy': mid, 'spec_high_energy': high}
    except Exception:
        return {'spec_centroid': np.nan, 'spec_domfreq': np.nan, 'spec_low_energy': np.nan, 'spec_mid_energy': np.nan, 'spec_high_energy': np.nan}

# augmentation transforms
def add_noise(x, rel_std):
    # x: T array
    rms = np.sqrt(np.mean(x**2))
    sigma = rel_std * (rms + 1e-12)
    return x + np.random.normal(0, sigma, size=x.shape)

def scale_amplitude(x, low=SCALE_MIN, high=SCALE_MAX):
    factor = np.random.uniform(low, high)
    return x * factor

def time_shift(x, max_shift):
    s = np.random.randint(-max_shift, max_shift+1)
    if s == 0:
        return x
    if s > 0:
        # shift right => pad left
        return np.pad(x[:-s], (s,0), mode='constant')
    else:
        s = abs(s)
        return np.pad(x[s:], (0,s), mode='constant')

def time_stretch_resample(x, factor):
    # simple resample using Fourier/resample
    if abs(factor - 1.0) < 1e-6:
        return x
    new_len = max(2, int(round(len(x) * factor)))
    return signal.resample(x, new_len)

def random_crop_and_pad(x, min_frac=CROP_MIN_FRAC):
    L = len(x)
    keep = int(np.round(np.random.uniform(min_frac, 1.0) * L))
    if keep >= L:
        return x
    start = np.random.randint(0, L - keep + 1)
    cropped = x[start:start+keep]
    # pad back to L with zeros
    pad_left = (L - keep)//2
    pad_right = L - keep - pad_left
    return np.pad(cropped, (pad_left, pad_right), mode='constant')

def small_filter(x, fs):
    # randomly apply a mild lowpass or highpass via FIR
    if np.random.rand() < 0.5:
        # lowpass: cutoff between 0.1*nyq and 0.4*nyq
        nyq = 0.5 * fs
        cutoff = np.random.uniform(0.05*nyq, 0.4*nyq)
        # design simple FIR with window
        from scipy.signal import firwin, lfilter
        taps = firwin(numtaps=31, cutoff=cutoff/(fs/2.0))
        return lfilter(taps, [1.0], x)
    else:
        # highpass small
        nyq = 0.5 * fs
        cutoff = np.random.uniform(0.02*nyq, 0.1*nyq)
        from scipy.signal import firwin, lfilter
        taps = firwin(numtaps=31, cutoff=cutoff/(fs/2.0), pass_zero=False)
        return lfilter(taps, [1.0], x)

# find the single npz window
files = sorted(glob.glob(os.path.join(NPZ_FOLDER, "*.npz")))
if len(files) == 0:
    raise SystemExit("No NPZ found in " + NPZ_FOLDER)
print("Found NPZ files:", files)
# pick first one (we know there is one)
z = np.load(files[0], allow_pickle=True)
V = z.get("V", None)
I = z.get("I", None)
t = z.get("t", None)
if V is None:
    raise SystemExit("No V in the NPZ.")
V = np.asarray(V)  # shape T x C
if V.ndim == 2 and V.shape[0] < V.shape[1]:
    V = V.T
T, C = V.shape
print("Original window shape (T,C):", V.shape)
if t is not None and len(t)>2:
    dt = np.median(np.diff(t.astype(float)))
    fs = 1.0/dt
else:
    fs = 10000.0
print("Estimated fs:", fs)

# create augmentation set
augmented = []
labels = []
filenames = []

# include original
if KEEP_ORIGINAL:
    augmented.append(V.copy())
    filenames.append(os.path.basename(files[0]) + "_orig")
    labels.append("Va")  # keep same detected_phase label

# generate N_AUG augmented examples by applying random chains of transforms
for i in range(N_AUG):
    V_aug = V.copy()  # T x C
    # for each channel apply slightly different transforms
    V_new = np.zeros_like(V_aug)
    for ch in range(C):
        ch_signal = V_aug[:, ch]
        # random sequence of transforms
        # 1) time stretch occasionally
        if np.random.rand() < 0.15:
            factor = np.random.uniform(TIME_STRETCH_MIN, TIME_STRETCH_MAX)
            s = time_stretch_resample(ch_signal, factor)
            # resample back to original length by trunc/pad or resample
            if len(s) != T:
                s = signal.resample(s, T)
            ch_signal = s
        # 2) crop and pad sometimes
        if np.random.rand() < 0.2:
            ch_signal = random_crop_and_pad(ch_signal, min_frac=CROP_MIN_FRAC)
        # 3) time shift
        if np.random.rand() < 0.4:
            ch_signal = time_shift(ch_signal, TIME_SHIFT_MAX_SAMPLES)
        # 4) amplitude scale
        if np.random.rand() < 0.6:
            ch_signal = scale_amplitude(ch_signal, SCALE_MIN, SCALE_MAX)
        # 5) filter
        if np.random.rand() < 0.3:
            ch_signal = small_filter(ch_signal, fs)
        # 6) additive noise
        if np.random.rand() < 0.9:
            ch_signal = add_noise(ch_signal, NOISE_STD_REL)
        V_new[:, ch] = ch_signal
    augmented.append(V_new)
    filenames.append(os.path.basename(files[0]) + f"_aug{i:04d}")
    labels.append("Va")

print("Total augmented samples created:", len(augmented))

# compute features for each augmented sample
rows = []
for idx, Vx in enumerate(augmented):
    feat = {}
    # ensure shape T x C
    if Vx.ndim == 1:
        Vx = Vx.reshape(-1,1)
    if Vx.shape[0] != T:
        # resample to T
        Vx = signal.resample(Vx, T)
    for ch in range(Vx.shape[1]):
        arr = Vx[:, ch].astype(float)
        tfs = time_features(arr)
        sfs = spectral_features(arr, fs)
        prefix = f"V{ch+1}"
        for k, v in tfs.items():
            feat[f"{prefix}_{k}"] = v
        for k, v in sfs.items():
            feat[f"{prefix}_{k}"] = v
    # cross-channel RMS stats if >=3 channels
    if Vx.shape[1] >= 3:
        rms_vals = [np.sqrt(np.mean(Vx[:,c]**2)) for c in range(3)]
        feat['V_rms_mean'] = float(np.mean(rms_vals))
        feat['V_rms_std'] = float(np.std(rms_vals))
        feat['V_rms_max'] = float(np.max(rms_vals))
        feat['V_rms_min'] = float(np.min(rms_vals))
    rows.append(feat)

X_df = pd.DataFrame(rows).fillna(0.0)
y = np.array([0 if lab.lower().startswith('v') or lab.lower().startswith('a') else 3 for lab in labels], dtype=int)
# This maps Va/A -> class 0. If you want to create synthetic classes for B/C, we can discuss generating them.
print("Feature matrix shape:", X_df.shape)
print("Label distribution:", {k:int((y==k).sum()) for k in np.unique(y)})

# Save augmented features table
feat_csv = os.path.join(OUT_DIR, "augmented_features.csv")
X_df['label'] = y
X_df['filename'] = filenames
X_df.to_csv(feat_csv, index=False)
print("Saved augmented features to", feat_csv)

# Prepare arrays
X = X_df.drop(columns=['label','filename']).values
scaler = StandardScaler()
Xs = scaler.fit_transform(X)

# train/val/test split stratified - if only one class present, we'll stratify by small artificial split to allow training
unique_classes = np.unique(y)
if len(unique_classes) == 1:
    print("Warning: only one class present (Va). Proceeding to split without stratify.")
    X_train, X_temp, y_train, y_temp = train_test_split(Xs, y, test_size=TEST_SIZE+VAL_SIZE, random_state=RANDOM_STATE)
    # split temp into val/test
    rel = TEST_SIZE / (TEST_SIZE + VAL_SIZE)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=rel, random_state=RANDOM_STATE)
else:
    idx = np.arange(len(y))
    idx_trainval, idx_test, y_trainval, y_test = train_test_split(idx, y, test_size=TEST_SIZE, stratify=y, random_state=RANDOM_STATE)
    val_frac = VAL_SIZE / (1.0 - TEST_SIZE)
    idx_train, idx_val, y_train, y_val = train_test_split(idx_trainval, y_trainval, test_size=val_frac, stratify=y_trainval, random_state=RANDOM_STATE)
    X_train, X_val, X_test = Xs[idx_train], Xs[idx_val], Xs[idx_test]
    y_train, y_val, y_test = y[idx_train], y[idx_val], y[idx_test]

print("Train/val/test sizes:", X_train.shape[0], X_val.shape[0], X_test.shape[0])

# If only one class exists, training a classifier is not meaningful; instead we will compute descriptive stats and save features.
if len(unique_classes) == 1:
    print("\nOnly one class available (Va). Training classifiers requires >=2 classes.")
    print("Saved augmented features; you can either (A) create synthetic examples for other phases, (B) re-run detection to get more events, or (C) perform unsupervised analysis.")
    # still save scaler & minimal artifacts
    joblib.dump(scaler, os.path.join(OUT_DIR, "scaler.joblib"))
    np.savez_compressed(os.path.join(OUT_DIR, "dataset_info.npz"), X_shape=X.shape, labels=y, filenames=np.array(filenames))
    print("Saved artifacts to", OUT_DIR)
    print("Step complete. If you want, I can now:")
    print("  1) generate synthetic B/C windows by perturbing phase relationships (needs rule-based approach),")
    print("  2) re-run detection more aggressively across the entire raw CSV to try to find more events, or")
    print("  3) show unsupervised clustering / anomaly-detection using the augmented set.")
    raise SystemExit(0)

# --- Fit ML models (only reached if >=2 classes) ---
def fit_eval_model(name, model):
    print("\n=== Model:", name, "===")
    try:
        cv = StratifiedKFold(n_splits=min(CV_FOLDS, max(2, len(y_train))), shuffle=True, random_state=RANDOM_STATE)
        scores = cross_val_score(model, X_train, y_train, cv=cv, scoring='accuracy', n_jobs=-1)
        print("CV accuracy (train): mean {:.4f} ± {:.4f}".format(scores.mean(), scores.std()))
    except Exception as e:
        print("CV failed:", e)
    model.fit(X_train, y_train)
    for split_name, Xsplt, ysplt in [("train", X_train, y_train), ("val", X_val, y_val), ("test", X_test, y_test)]:
        preds = model.predict(Xsplt)
        acc = accuracy_score(ysplt, preds)
        print(f"{split_name} accuracy: {acc:.4f}")
        if split_name == "test":
            print("\nClassification report (test):")
            print(classification_report(ysplt, preds, digits=4))
            print("Confusion matrix (test):")
            print(confusion_matrix(ysplt, preds))
    joblib.dump(model, os.path.join(OUT_DIR, f"{name.replace(' ','_')}.joblib"))
    print("Saved model to", os.path.join(OUT_DIR, f"{name.replace(' ','_')}.joblib"))

# Random Forest always available
rf = RandomForestClassifier(n_estimators=300, random_state=RANDOM_STATE, n_jobs=-1)
fit_eval_model("RandomForest", rf)

# XGBoost
if xgb is not None:
    xclf = xgb.XGBClassifier(use_label_encoder=False, eval_metric="mlogloss", random_state=RANDOM_STATE, n_jobs=-1)
    fit_eval_model("XGBoost", xclf)
else:
    print("\nXGBoost not available; skipped.")

# LightGBM
if lgb is not None:
    lclf = lgb.LGBMClassifier(random_state=RANDOM_STATE, n_jobs=-1)
    fit_eval_model("LightGBM", lclf)
else:
    print("\nLightGBM not available; skipped.")

# CatBoost
if CatBoostClassifier is not None:
    try:
        cclf = CatBoostClassifier(verbose=0, random_state=RANDOM_STATE)
        fit_eval_model("CatBoost", cclf)
    except Exception as e:
        print("CatBoost failed:", e)
else:
    print("\nCatBoost not available; skipped.")

# Save scaler & feature names
joblib.dump(scaler, os.path.join(OUT_DIR, "scaler.joblib"))
pd.DataFrame({'feature': X_df.columns.tolist()}).to_csv(os.path.join(OUT_DIR, "feature_names.csv"), index=False)
np.savez_compressed(os.path.join(OUT_DIR, "dataset_info.npz"), X_shape=X.shape, labels=y, filenames=np.array(filenames))

print("\nAll done. Artifacts saved to", OUT_DIR)

Found NPZ files: ['windows_outputs_robust/npz\\window_000_onsetidx2141.npz']
Original window shape (T,C): (2201, 3)
Estimated fs: 10000.0000000011
Total augmented samples created: 401
Feature matrix shape: (401, 52)
Label distribution: {0: 401}
Saved augmented features to ml_aug_out\augmented_features.csv
Train/val/test sizes: 280 60 61

Only one class available (Va). Training classifiers requires >=2 classes.
Saved augmented features; you can either (A) create synthetic examples for other phases, (B) re-run detection to get more events, or (C) perform unsupervised analysis.
Saved artifacts to ml_aug_out
Step complete. If you want, I can now:
  1) generate synthetic B/C windows by perturbing phase relationships (needs rule-based approach),
  2) re-run detection more aggressively across the entire raw CSV to try to find more events, or
  3) show unsupervised clustering / anomaly-detection using the augmented set.


SystemExit: 0

In [19]:
# step4_synth_BC_and_train.py
import os, glob, math, random, warnings
warnings.filterwarnings("ignore")
import numpy as np, pandas as pd
from scipy import stats, signal
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import joblib

# optional boosters
try:
    import xgboost as xgb
except Exception:
    xgb = None
try:
    import lightgbm as lgb
except Exception:
    lgb = None
try:
    from catboost import CatBoostClassifier
except Exception:
    CatBoostClassifier = None

# === CONFIG ===
ORIG_NPZ = "windows_outputs_robust/npz/window_000_onsetidx2141.npz"
OUT_NPZ_DIR = "synthetic_npz"
OUT_DIR = "ml_synth_out"
os.makedirs(OUT_NPZ_DIR, exist_ok=True)
os.makedirs(OUT_DIR, exist_ok=True)

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)
random.seed(RANDOM_STATE)

N_AUG = 400      # number of augmented Va variants to generate (per original)
KEEP_ORIGINAL = True

# augmentation params (same as previous)
NOISE_STD_REL = 0.02
SCALE_MIN, SCALE_MAX = 0.85, 1.15
TIME_SHIFT_MAX_SAMPLES = 60
TIME_STRETCH_MIN, TIME_STRETCH_MAX = 0.95, 1.05
CROP_MIN_FRAC = 0.85

# helper functions
from scipy.signal import welch, resample, firwin, lfilter

def time_features(ch):
    f = {}
    f['mean'] = np.mean(ch)
    f['std'] = np.std(ch)
    f['rms'] = np.sqrt(np.mean(ch**2))
    f['ptp'] = np.ptp(ch)
    f['abs_mean'] = np.mean(np.abs(ch))
    f['skew'] = float(stats.skew(ch))
    f['kurtosis'] = float(stats.kurtosis(ch))
    f['median'] = np.median(ch)
    f['max'] = np.max(ch)
    f['min'] = np.min(ch)
    f['energy'] = np.sum(ch**2)
    return f

def spectral_features(ch, fs):
    try:
        f, Pxx = welch(ch, fs=fs, nperseg=min(256, len(ch)))
        sc = np.sum(f * Pxx) / (np.sum(Pxx) + 1e-12)
        dom = f[np.argmax(Pxx)]
        nyq = fs/2.0
        low = np.sum(Pxx[(f>=0) & (f < 0.1*nyq)])
        mid = np.sum(Pxx[(f>=0.1*nyq) & (f < 0.4*nyq)])
        high = np.sum(Pxx[(f>=0.4*nyq)])
        return {'spec_centroid': sc, 'spec_domfreq': dom, 'spec_low_energy': low, 'spec_mid_energy': mid, 'spec_high_energy': high}
    except Exception:
        return {'spec_centroid': np.nan, 'spec_domfreq': np.nan, 'spec_low_energy': np.nan, 'spec_mid_energy': np.nan, 'spec_high_energy': np.nan}

# transforms on a single channel
def add_noise(x, rel_std):
    rms = np.sqrt(np.mean(x**2)) + 1e-12
    sigma = rel_std * rms
    return x + np.random.normal(0, sigma, size=x.shape)

def scale_amplitude(x, low=SCALE_MIN, high=SCALE_MAX):
    return x * np.random.uniform(low, high)

def time_shift(x, max_shift):
    s = np.random.randint(-max_shift, max_shift+1)
    if s == 0:
        return x
    if s > 0:
        return np.pad(x[:-s], (s,0), mode='constant')
    else:
        s = abs(s)
        return np.pad(x[s:], (0,s), mode='constant')

def time_stretch(x, factor):
    if abs(factor-1.0) < 1e-6:
        return x
    new_len = max(2, int(round(len(x)*factor)))
    return resample(x, new_len)

def random_crop_and_pad(x, min_frac=CROP_MIN_FRAC):
    L = len(x)
    keep = int(np.round(np.random.uniform(min_frac, 1.0) * L))
    if keep >= L:
        return x
    start = np.random.randint(0, L-keep+1)
    cropped = x[start:start+keep]
    pad_left = (L - keep)//2
    pad_right = L - keep - pad_left
    return np.pad(cropped, (pad_left, pad_right), mode='constant')

def mild_filter(x, fs):
    # lowpass small prob, else identity
    if np.random.rand() < 0.4:
        cutoff = np.random.uniform(0.05*(fs/2.0), 0.4*(fs/2.0))
        taps = firwin(numtaps=31, cutoff=cutoff/(fs/2.0))
        return lfilter(taps, [1.0], x)
    return x

# load original NPZ
if not os.path.isfile(ORIG_NPZ):
    raise SystemExit(f"Original NPZ not found: {ORIG_NPZ}")
z = np.load(ORIG_NPZ, allow_pickle=True)
V = z.get("V", None)   # shape T x 3
I = z.get("I", None)
t = z.get("t", None)
if V is None:
    raise SystemExit("No V found in original NPZ.")
V = np.asarray(V)
if V.ndim == 2 and V.shape[0] < V.shape[1]:
    V = V.T
T, C = V.shape
if t is not None and len(t)>2:
    dt = np.median(np.diff(t.astype(float)))
    fs = 1.0/dt
else:
    fs = 10000.0

print("Loaded original NPZ:", ORIG_NPZ)
print("Original V shape (T,C):", V.shape, "fs:", fs)

# produce augmented Va samples (time-series) first
va_samples = []
filenames = []

if KEEP_ORIGINAL:
    va_samples.append(V.copy())
    filenames.append("Va_orig")

for i in range(N_AUG):
    Vnew = np.zeros_like(V)
    for ch in range(C):
        x = V[:, ch].copy()
        # random small stretch
        if np.random.rand() < 0.12:
            fct = np.random.uniform(TIME_STRETCH_MIN, TIME_STRETCH_MAX)
            xs = time_stretch(x, fct)
            if len(xs) != T:
                xs = resample(xs, T)
            x = xs
        # random crop/pad
        if np.random.rand() < 0.18:
            x = random_crop_and_pad(x)
        # shift
        if np.random.rand() < 0.5:
            x = time_shift(x, TIME_SHIFT_MAX_SAMPLES)
        # scale
        if np.random.rand() < 0.6:
            x = scale_amplitude(x)
        # filter
        if np.random.rand() < 0.3:
            x = mild_filter(x, fs)
        # noise
        if np.random.rand() < 0.9:
            x = add_noise(x, NOISE_STD_REL)
        Vnew[:, ch] = x
    va_samples.append(Vnew)
    filenames.append(f"Va_aug{i:04d}")

print("Generated Va augmented samples:", len(va_samples))

# For each Va sample, generate B and C by rotating channels and applying slight extra perturbations
all_samples = []
all_labels = []
all_fnames = []

def rotate_forward(Vx):
    # Va->Vb, Vb->Vc, Vc->Va (columns)
    return np.roll(Vx, -1, axis=1)

def rotate_backward(Vx):
    return np.roll(Vx, 1, axis=1)

for i, Vx in enumerate(va_samples):
    # class A: original/augmented
    all_samples.append(Vx)
    all_labels.append(0)  # 0 -> A (Va)
    all_fnames.append(filenames[i])

    # class B: rotate forward + small extra jitter (time shift + scale)
    Vb = rotate_forward(Vx.copy())
    for ch in range(C):
        if np.random.rand() < 0.5:
            Vb[:,ch] = time_shift(Vb[:,ch], max_shift=TIME_SHIFT_MAX_SAMPLES//2)
        if np.random.rand() < 0.6:
            Vb[:,ch] = scale_amplitude(Vb[:,ch], low=0.95, high=1.05)
        if np.random.rand() < 0.4:
            Vb[:,ch] = add_noise(Vb[:,ch], rel_std=NOISE_STD_REL*1.1)
    all_samples.append(Vb)
    all_labels.append(1)  # 1 -> B
    all_fnames.append(filenames[i] + "_rotF")

    # class C: rotate backward + slightly different perturb
    Vc = rotate_backward(Vx.copy())
    for ch in range(C):
        if np.random.rand() < 0.5:
            Vc[:,ch] = time_shift(Vc[:,ch], max_shift=TIME_SHIFT_MAX_SAMPLES//2)
        if np.random.rand() < 0.6:
            Vc[:,ch] = scale_amplitude(Vc[:,ch], low=0.95, high=1.05)
        if np.random.rand() < 0.4:
            Vc[:,ch] = add_noise(Vc[:,ch], rel_std=NOISE_STD_REL*1.2)
    all_samples.append(Vc)
    all_labels.append(2)  # 2 -> C
    all_fnames.append(filenames[i] + "_rotB")

# sanity
print("Total synthesized samples (A/B/C):", len(all_samples), "labels distribution:", {k:int(all_labels.count(k)) for k in set(all_labels)})

# save time-series npz for future inspection
for idx, (Vx, lbl, fname) in enumerate(zip(all_samples, all_labels, all_fnames)):
    outname = os.path.join(OUT_NPZ_DIR, f"{idx:05d}_{fname}_lbl{lbl}.npz")
    # also create synthetic currents by rotating original I similarly if present
    if I is not None:
        I_arr = np.asarray(I)
        if I_arr.ndim == 2 and I_arr.shape[0] < I_arr.shape[1]:
            I_arr = I_arr.T
        Ilen = I_arr.shape[0]
        # create synthetic currents by rotating columns + small noise
        Ic = np.roll(I_arr, -1 if lbl==1 else 1 if lbl==2 else 0, axis=1)
        # add tiny noise
        Ic = Ic + np.random.normal(0, 0.01*np.std(Ic), size=Ic.shape)
        np.savez_compressed(outname, V=Vx, I=Ic)
    else:
        np.savez_compressed(outname, V=Vx)
# Save a CSV index
pd.DataFrame({'npz_file': sorted(glob.glob(os.path.join(OUT_NPZ_DIR,"*.npz"))),
              'label': all_labels,
              'fname': all_fnames}).to_csv(os.path.join(OUT_DIR, "synth_index.csv"), index=False)
print("Saved synthetic time-series to", OUT_NPZ_DIR)

# === compute features for all synthetic windows ===
rows = []
labels = []
filenames = []
for f in sorted(glob.glob(os.path.join(OUT_NPZ_DIR,"*.npz"))):
    z = np.load(f, allow_pickle=True)
    Vx = z.get("V", None)
    Icx = z.get("I", None)
    if Vx is None:
        continue
    Vx = np.asarray(Vx)
    if Vx.ndim == 2 and Vx.shape[0] < Vx.shape[1]:
        Vx = Vx.T
    # ensure T length same as original by resampling if needed
    if Vx.shape[0] != T:
        Vx = resample(Vx, T)
    feat = {}
    for ch in range(Vx.shape[1]):
        arr = Vx[:, ch].astype(float)
        tfs = time_features(arr)
        sfs = spectral_features(arr, fs)
        pref = f"V{ch+1}"
        for k,v in tfs.items(): feat[f"{pref}_{k}"] = v
        for k,v in sfs.items(): feat[f"{pref}_{k}"] = v
    if Vx.shape[1] >= 3:
        rms_vals = [np.sqrt(np.mean(Vx[:,c]**2)) for c in range(3)]
        feat['V_rms_mean'] = float(np.mean(rms_vals))
        feat['V_rms_std'] = float(np.std(rms_vals))
    # parse label from filename (lblX)
    base = os.path.basename(f)
    lbl = None
    if "_lbl0" in base or "lbl0" in base:
        lbl = 0
    elif "_lbl1" in base or "lbl1" in base:
        lbl = 1
    elif "_lbl2" in base or "lbl2" in base:
        lbl = 2
    else:
        # fallback by reading index file
        lbl = None
    rows.append(feat)
    labels.append(lbl)
    filenames.append(base)

X_df = pd.DataFrame(rows).fillna(0.0)
y = np.array(labels, dtype=int)
print("Feature matrix shape:", X_df.shape)
print("Label distribution:", {k:int((y==k).sum()) for k in np.unique(y)})

# save features table
feat_csv = os.path.join(OUT_DIR, "synth_features.csv")
X_df['label'] = y
X_df['filename'] = filenames
X_df.to_csv(feat_csv, index=False)
print("Saved feature table to", feat_csv)

# Prepare arrays & scale
X = X_df.drop(columns=['label','filename']).values
scaler = StandardScaler()
Xs = scaler.fit_transform(X)

# train/val/test split stratified
idx = np.arange(len(y))
idx_trainval, idx_test, y_trainval, y_test = train_test_split(idx, y, test_size=0.15, stratify=y, random_state=RANDOM_STATE)
idx_train, idx_val, y_train, y_val = train_test_split(idx_trainval, y_trainval, test_size=0.17647058823529413, stratify=y_trainval, random_state=RANDOM_STATE)
# (the val fraction chosen so final splits are ~70/15/15)
X_train, X_val, X_test = Xs[idx_train], Xs[idx_val], Xs[idx_test]
y_train, y_val, y_test = y[idx_train], y[idx_val], y[idx_test]
print("Split sizes train/val/test:", X_train.shape[0], X_val.shape[0], X_test.shape[0])

# Function to fit + eval
def fit_eval_model(name, model):
    print("\n=== Model:", name, "===")
    try:
        cv = StratifiedKFold(n_splits=min(5, max(2,len(y_train))), shuffle=True, random_state=RANDOM_STATE)
        scores = cross_val_score(model, X_train, y_train, cv=cv, scoring='accuracy', n_jobs=-1)
        print("CV acc (train): {:.4f} ± {:.4f}".format(scores.mean(), scores.std()))
    except Exception as e:
        print("CV failed:", e)
    model.fit(X_train, y_train)
    for split_name, Xsplt, ysplt in [("train", X_train, y_train), ("val", X_val, y_val), ("test", X_test, y_test)]:
        preds = model.predict(Xsplt)
        acc = accuracy_score(ysplt, preds)
        print(f"{split_name} accuracy: {acc:.4f}")
        if split_name == "test":
            print("\nClassification report (test):")
            print(classification_report(ysplt, preds, digits=4))
            print("Confusion matrix (test):")
            print(confusion_matrix(ysplt, preds))
    joblib.dump(model, os.path.join(OUT_DIR, f"{name.replace(' ','_')}.joblib"))
    print("Saved model to", os.path.join(OUT_DIR, f"{name.replace(' ','_')}.joblib"))

# train RandomForest
rf = RandomForestClassifier(n_estimators=300, random_state=RANDOM_STATE, n_jobs=-1)
fit_eval_model("RandomForest", rf)

# XGBoost
if xgb is not None:
    xclf = xgb.XGBClassifier(use_label_encoder=False, eval_metric="mlogloss", random_state=RANDOM_STATE, n_jobs=-1)
    fit_eval_model("XGBoost", xclf)
else:
    print("\nXGBoost not available; skipped.")

# LightGBM
if lgb is not None:
    lclf = lgb.LGBMClassifier(random_state=RANDOM_STATE, n_jobs=-1)
    fit_eval_model("LightGBM", lclf)
else:
    print("\nLightGBM not available; skipped.")

# save scaler & feature names
joblib.dump(scaler, os.path.join(OUT_DIR, "scaler.joblib"))
pd.DataFrame({'feature': X_df.drop(columns=['label','filename']).columns.tolist()}).to_csv(os.path.join(OUT_DIR, "feature_names.csv"), index=False)
np.savez_compressed(os.path.join(OUT_DIR, "dataset_info.npz"), X_shape=X.shape, labels=y, filenames=np.array(filenames))

print("\nAll done. Artifacts saved to", OUT_DIR, "and time-series to", OUT_NPZ_DIR)
print("Please paste the first ~120 lines of console output and confirm files exist in", OUT_DIR)

Loaded original NPZ: windows_outputs_robust/npz/window_000_onsetidx2141.npz
Original V shape (T,C): (2201, 3) fs: 10000.0000000011
Generated Va augmented samples: 401
Total synthesized samples (A/B/C): 1203 labels distribution: {0: 401, 1: 401, 2: 401}
Saved synthetic time-series to synthetic_npz
Feature matrix shape: (1203, 50)
Label distribution: {0: 401, 1: 401, 2: 401}
Saved feature table to ml_synth_out\synth_features.csv
Split sizes train/val/test: 841 181 181

=== Model: RandomForest ===
CV acc (train): 1.0000 ± 0.0000
train accuracy: 1.0000
val accuracy: 1.0000
test accuracy: 1.0000

Classification report (test):
              precision    recall  f1-score   support

           0     1.0000    1.0000    1.0000        60
           1     1.0000    1.0000    1.0000        60
           2     1.0000    1.0000    1.0000        61

    accuracy                         1.0000       181
   macro avg     1.0000    1.0000    1.0000       181
weighted avg     1.0000    1.0000    1.0000  

In [20]:
# step5_feature_importance_shap.py
import os, glob, warnings, joblib
warnings.filterwarnings("ignore")
import numpy as np, pandas as pd
import matplotlib.pyplot as plt

from sklearn.inspection import permutation_importance
from sklearn.ensemble import RandomForestClassifier

ML_DIR = "ml_synth_out"
PLOTS_DIR = os.path.join(ML_DIR, "plots")
os.makedirs(PLOTS_DIR, exist_ok=True)

# load model & features
model_path = os.path.join(ML_DIR, "RandomForest.joblib")
feat_csv = os.path.join(ML_DIR, "synth_features.csv")
if not os.path.isfile(model_path):
    raise SystemExit(f"RandomForest model not found at {model_path}")
if not os.path.isfile(feat_csv):
    raise SystemExit(f"Feature CSV not found at {feat_csv}")

print("Loading model and features...")
clf = joblib.load(model_path)
df = pd.read_csv(feat_csv)
print("Feature table shape:", df.shape)
if 'label' not in df.columns:
    raise SystemExit("feature table must contain 'label' column")

X = df.drop(columns=['label','filename'], errors='ignore')
y = df['label'].values
feature_names = X.columns.tolist()
X_np = X.values

# 1) impurity importances
try:
    imp = clf.feature_importances_
    idx_sorted = np.argsort(imp)[::-1]
    top_k = min(25, len(feature_names))
    top_idx = idx_sorted[:top_k]
    top_feats = [(feature_names[i], float(imp[i])) for i in top_idx]
    fi_df = pd.DataFrame(top_feats, columns=['feature','importance'])
    fi_df.to_csv(os.path.join(ML_DIR, "feature_importances_mdi.csv"), index=False)
    print("Saved MDI importances to", os.path.join(ML_DIR, "feature_importances_mdi.csv"))

    # bar plot
    plt.figure(figsize=(8, max(4, 0.2*top_k)))
    plt.barh(range(top_k)[::-1], [imp[i] for i in top_idx], align='center')
    plt.yticks(range(top_k)[::-1], [feature_names[i] for i in top_idx])
    plt.xlabel("MDI feature importance")
    plt.title("Top-{} RandomForest feature importances (MDI)".format(top_k))
    plt.tight_layout()
    outpng = os.path.join(PLOTS_DIR, "mdi_feature_importances.png")
    plt.savefig(outpng, dpi=200)
    plt.close()
    print("Saved MDI plot to", outpng)
except Exception as e:
    print("MDI importances failed:", e)

# 2) permutation importance (robust)
print("Computing permutation importances (this may take a minute)...")
try:
    r = permutation_importance(clf, X_np, y, n_repeats=12, random_state=42, n_jobs=-1, scoring='accuracy')
    perm_imp = r.importances_mean
    idx_sorted_p = np.argsort(perm_imp)[::-1]
    top_k = min(25, len(feature_names))
    top_idx_p = idx_sorted_p[:top_k]
    perm_df = pd.DataFrame([(feature_names[i], float(perm_imp[i])) for i in top_idx_p], columns=['feature','perm_importance'])
    perm_df.to_csv(os.path.join(ML_DIR, "feature_importances_permutation.csv"), index=False)
    print("Saved permutation importances to", os.path.join(ML_DIR, "feature_importances_permutation.csv"))

    plt.figure(figsize=(8, max(4, 0.2*top_k)))
    plt.barh(range(top_k)[::-1], [perm_imp[i] for i in top_idx_p], align='center')
    plt.yticks(range(top_k)[::-1], [feature_names[i] for i in top_idx_p])
    plt.xlabel("Permutation importance (accuracy drop)")
    plt.title("Top-{} permutation feature importances".format(top_k))
    plt.tight_layout()
    outpng2 = os.path.join(PLOTS_DIR, "perm_feature_importances.png")
    plt.savefig(outpng2, dpi=200)
    plt.close()
    print("Saved permutation plot to", outpng2)
except Exception as e:
    print("Permutation importance failed:", e)

# 3) SHAP (if installed) — uses TreeExplainer for RF
print("Trying SHAP explanations (if 'shap' installed)...")
try:
    import shap
    expl = shap.TreeExplainer(clf)
    # sample subset for speed
    sample_idx = np.random.choice(len(X_np), min(200, len(X_np)), replace=False)
    X_sample = X_np[sample_idx]
    shap_values = expl.shap_values(X_sample)
    # summary plot
    shap.summary_plot(shap_values, X_sample, feature_names=feature_names, show=False, plot_size=(8,6))
    shap_png = os.path.join(PLOTS_DIR, "shap_summary.png")
    plt.savefig(shap_png, dpi=200)
    plt.close()
    print("Saved SHAP summary to", shap_png)
    # save shap values for further analysis
    np.savez_compressed(os.path.join(ML_DIR, "shap_values.npz"), shap_values=shap_values, sample_idx=sample_idx)
    print("Saved SHAP values.")
except Exception as e:
    print("SHAP not available or failed:", e)
    print("You can install shap via: pip install shap")

print("\nStep complete. Plots & CSVs saved to:", ML_DIR, "and", PLOTS_DIR)
print("Please paste the printed console output and confirm the files were created:")
print(" -", os.path.join(ML_DIR, "feature_importances_mdi.csv"))
print(" -", os.path.join(ML_DIR, "feature_importances_permutation.csv"))
print(" -", os.path.join(PLOTS_DIR, "mdi_feature_importances.png"))
print(" -", os.path.join(PLOTS_DIR, "perm_feature_importances.png"))
print(" - (optional) SHAP file:", os.path.join(PLOTS_DIR, "shap_summary.png"))

Loading model and features...
Feature table shape: (1203, 52)
Saved MDI importances to ml_synth_out\feature_importances_mdi.csv
Saved MDI plot to ml_synth_out\plots\mdi_feature_importances.png
Computing permutation importances (this may take a minute)...
Saved permutation importances to ml_synth_out\feature_importances_permutation.csv
Saved permutation plot to ml_synth_out\plots\perm_feature_importances.png
Trying SHAP explanations (if 'shap' installed)...
Saved SHAP summary to ml_synth_out\plots\shap_summary.png
Saved SHAP values.

Step complete. Plots & CSVs saved to: ml_synth_out and ml_synth_out\plots
Please paste the printed console output and confirm the files were created:
 - ml_synth_out\feature_importances_mdi.csv
 - ml_synth_out\feature_importances_permutation.csv
 - ml_synth_out\plots\mdi_feature_importances.png
 - ml_synth_out\plots\perm_feature_importances.png
 - (optional) SHAP file: ml_synth_out\plots\shap_summary.png


<Figure size 768x576 with 0 Axes>

In [25]:
# catboost_baseline_fixed.py
import os
import numpy as np
import pandas as pd
import joblib
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import matplotlib.pyplot as plt

CSV_PATH = "merged_dataset.csv"
OUT_DIR = "catboost_outputs"
os.makedirs(OUT_DIR, exist_ok=True)

# --- Load ---
df = pd.read_csv(CSV_PATH)
print("Loaded:", CSV_PATH, " shape:", df.shape)
print("Columns:", df.columns.tolist())

if "Fault" not in df.columns:
    raise SystemExit("Column 'Fault' not found. Please ensure column exists.")

# show unique labels (including NaN handling)
vals = df["Fault"].dropna().unique()
print("\nOriginal Fault unique values (non-NaN):", np.sort(vals))

# decide labeling strategy
unique_vals = np.unique(df["Fault"].astype(str).fillna("NA"))
# handle numeric labels cleanly
fault_unique = pd.unique(df["Fault"].values)
n_unique = len(fault_unique[~pd.isnull(fault_unique)])
print("Count of unique Fault values (excluding NaN):", n_unique)

# If only one unique label, overwrite Fault with synthetic binary (clean)
if n_unique <= 1:
    print("\n⚠️ Only one unique Fault label found — creating synthetic binary labels (overwrite).")
    n = len(df)
    # synthetic: label first 50% as normal (0), last 50% as fault (1)
    split_idx = n // 2
    df = df.copy()
    df["Fault"] = 0
    df.loc[split_idx:, "Fault"] = 1
    print(f"Created synthetic labels: 0 for rows [0:{split_idx-1}], 1 for rows [{split_idx}:{n-1}]")
else:
    print("\nDataset already contains multiple Fault labels — will use them as-is.")

# After possible overwrite, prepare features & target
df = df.dropna(axis=0, subset=["Fault"])  # drop rows with missing target if any
y = df["Fault"].astype(int).values
features = [c for c in df.columns.tolist() if c != "Fault"]

print("\nFinal label distribution:")
(unique, counts) = np.unique(y, return_counts=True)
for u, c in zip(unique, counts):
    print(f"  label={u}  count={c}")

# choose objective
if len(unique) == 2:
    loss = "Logloss"
    eval_metric = "Accuracy"
    print("\nUsing binary objective (Logloss).")
else:
    loss = "MultiClass"
    eval_metric = "MultiClass"
    print("\nUsing multiclass objective (MultiClass).")

# train/test split (stratify if possible)
if len(unique) >= 2:
    X_train, X_test, y_train, y_test = train_test_split(
        df[features].values, y, test_size=0.15, random_state=42, stratify=y
    )
else:
    X_train, X_test, y_train, y_test = train_test_split(
        df[features].values, y, test_size=0.15, random_state=42
    )

print(f"\nTrain samples: {X_train.shape[0]} | Test samples: {X_test.shape[0]} | Features: {X_train.shape[1]}")

# --- Train CatBoost ---
model = CatBoostClassifier(
    iterations=500,
    learning_rate=0.05,
    depth=8,
    loss_function=loss,
    eval_metric=eval_metric,
    random_seed=42,
    verbose=100
)

train_pool = Pool(X_train, y_train)
test_pool = Pool(X_test, y_test)

print("\nTraining CatBoost... (you will see periodic logs)")
model.fit(train_pool, eval_set=test_pool, use_best_model=True)

# --- Evaluate ---
y_pred = model.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print(f"\nTest Accuracy: {acc:.4f}\n")
print("Classification Report:")
print(classification_report(y_test, y_pred, digits=4))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

# --- Feature importances ---
feat_imp = model.get_feature_importance()
feat_names = features
imp_df = pd.DataFrame({"Feature": feat_names, "Importance": feat_imp}).sort_values("Importance", ascending=False)
imp_csv = os.path.join(OUT_DIR, "feature_importance.csv")
imp_df.to_csv(imp_csv, index=False)
print("\nSaved feature importances to:", imp_csv)

# plot top 15
topk = min(15, len(feat_names))
plt.figure(figsize=(8, max(4, 0.35*topk)))
plt.barh(imp_df["Feature"].head(topk)[::-1], imp_df["Importance"].head(topk)[::-1])
plt.xlabel("Importance")
plt.title("Top-{} CatBoost Feature Importances".format(topk))
plt.tight_layout()
png_path = os.path.join(OUT_DIR, "feature_importance_top15.png")
plt.savefig(png_path, dpi=200)
plt.close()
print("Saved feature importance plot to:", png_path)

# save model
model_path = os.path.join(OUT_DIR, "catboost_model.cbm")
model.save_model(model_path)
joblib.dump(model, os.path.join(OUT_DIR, "catboost_model.joblib"))
print("Saved CatBoost model to:", model_path)

print("\nDone. All outputs in folder:", OUT_DIR)

Loaded: merged_dataset.csv  shape: (16004, 8)
Columns: ['t', 'Va', 'Vb', 'Vc', 'Ia', 'Ib', 'Ic', 'Fault']

Original Fault unique values (non-NaN): [0 1 2 3]
Count of unique Fault values (excluding NaN): 4

Dataset already contains multiple Fault labels — will use them as-is.

Final label distribution:
  label=0  count=10001
  label=1  count=2001
  label=2  count=2001
  label=3  count=2001

Using multiclass objective (MultiClass).

Train samples: 13603 | Test samples: 2401 | Features: 7

Training CatBoost... (you will see periodic logs)
0:	learn: 1.2669735	test: 1.2680282	best: 1.2680282 (0)	total: 298ms	remaining: 2m 28s
100:	learn: 0.0420794	test: 0.0411255	best: 0.0411255 (100)	total: 7.13s	remaining: 28.2s
200:	learn: 0.0145262	test: 0.0135927	best: 0.0135927 (200)	total: 13.5s	remaining: 20s
300:	learn: 0.0075521	test: 0.0070984	best: 0.0070984 (300)	total: 19.8s	remaining: 13.1s
400:	learn: 0.0048061	test: 0.0045804	best: 0.0045804 (400)	total: 25.6s	remaining: 6.32s
499:	learn: 0

In [26]:
# ============================================================
#  CATBOOST BASELINE (IEEE PUBLICATION READY)
# ============================================================
import os
import numpy as np
import pandas as pd
import joblib
import matplotlib.pyplot as plt
import seaborn as sns
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    classification_report, confusion_matrix, accuracy_score,
    roc_curve, auc, precision_recall_curve, average_precision_score
)
from sklearn.preprocessing import label_binarize

# === CONFIG ===
CSV_PATH = "merged_dataset.csv"
OUT_DIR = "catboost_outputs"
os.makedirs(OUT_DIR, exist_ok=True)

# === LOAD DATA ===
df = pd.read_csv(CSV_PATH)
print("Loaded:", CSV_PATH, " shape:", df.shape)
if "Fault" not in df.columns:
    raise SystemExit("Column 'Fault' not found in dataset.")

# Handle single-label datasets
vals = df["Fault"].dropna().unique()
print("\nOriginal Fault labels:", np.sort(vals))
n_unique = len([v for v in vals if str(v) != "nan"])
if n_unique <= 1:
    print("\n⚠️ Only one unique label found — creating synthetic binary labels.")
    n = len(df)
    half = n // 2
    df["Fault"] = 0
    df.loc[half:, "Fault"] = 1
    print(f"Created synthetic 0/1 labels (0: first {half}, 1: rest).")
else:
    print("\nMultiple unique labels found — using as-is.")

# === FEATURES / TARGET ===
df = df.dropna(subset=["Fault"])
y = df["Fault"].astype(int).values
features = [c for c in df.columns if c != "Fault"]
X = df[features].values
print("\nFinal label distribution:", np.unique(y, return_counts=True))

# === SPLIT ===
strat = y if len(np.unique(y)) > 1 else None
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.15, random_state=42, stratify=strat
)
print(f"\nTrain={len(X_train)}, Test={len(X_test)}, Features={X.shape[1]}")

# === TRAIN MODEL ===
loss = "Logloss" if len(np.unique(y)) == 2 else "MultiClass"
eval_metric = "Accuracy" if len(np.unique(y)) == 2 else "MultiClass"

model = CatBoostClassifier(
    iterations=500,
    learning_rate=0.05,
    depth=8,
    loss_function=loss,
    eval_metric=eval_metric,
    random_seed=42,
    verbose=100
)
train_pool = Pool(X_train, y_train)
test_pool = Pool(X_test, y_test)
print("\nTraining CatBoost...")
model.fit(train_pool, eval_set=test_pool, use_best_model=True)

# === EVALUATION ===
y_pred = model.predict(X_test)
y_prob = model.predict_proba(X_test)
acc = accuracy_score(y_test, y_pred)
print(f"\n✅ Test Accuracy: {acc:.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred, digits=4))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

# === SAVE MODEL ===
model.save_model(os.path.join(OUT_DIR, "catboost_model.cbm"))
joblib.dump(model, os.path.join(OUT_DIR, "catboost_model.joblib"))

# === FEATURE IMPORTANCE ===
feat_imp = model.get_feature_importance(train_pool)
imp_df = pd.DataFrame({"Feature": features, "Importance": feat_imp}).sort_values("Importance", ascending=False)
imp_df.to_csv(os.path.join(OUT_DIR, "feature_importance.csv"), index=False)

# ============================================================
#  IEEE FIGURE SETTINGS
# ============================================================
plt.rcParams.update({
    "font.family": "serif",
    "font.serif": ["Times New Roman", "Times", "DejaVu Serif", "Nimbus Roman No9 L"],
    "font.size": 12,
    "axes.labelsize": 12,
    "axes.titlesize": 14,
    "axes.labelweight": "bold",
    "axes.titleweight": "bold",
    "xtick.labelsize": 12,
    "ytick.labelsize": 12,
    "legend.fontsize": 11,
    "figure.dpi": 120,
    "savefig.dpi": 300,
    "axes.grid": True,
    "grid.alpha": 0.3,
    "grid.linestyle": "--",
    "savefig.bbox": "tight",
    "savefig.pad_inches": 0.05
})

# ============================================================
# 1️⃣ CONFUSION MATRIX
# ============================================================
cm = confusion_matrix(y_test, y_pred)
labels = [str(u) for u in np.unique(y_test)]
fig, ax = plt.subplots(figsize=(5, 4))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", cbar=False,
            xticklabels=labels, yticklabels=labels, annot_kws={"size": 11})
ax.set_xlabel("Predicted Label", fontweight="bold")
ax.set_ylabel("True Label", fontweight="bold")
ax.set_title("CatBoost Confusion Matrix", fontweight="bold")
plt.tight_layout()
plt.savefig(os.path.join(OUT_DIR, "confusion_matrix_ieee.png"))
plt.savefig(os.path.join(OUT_DIR, "confusion_matrix_ieee.pdf"))
plt.close()

# ============================================================
# 2️⃣ ROC CURVES
# ============================================================
n_classes = len(np.unique(y_test))
fig, ax = plt.subplots(figsize=(6, 5))
if n_classes == 2:
    fpr, tpr, _ = roc_curve(y_test, y_prob[:, 1])
    roc_auc = auc(fpr, tpr)
    ax.plot(fpr, tpr, lw=2, label=f"ROC (AUC = {roc_auc:.3f})")
else:
    y_bin = label_binarize(y_test, classes=np.unique(y_test))
    for i in range(n_classes):
        fpr, tpr, _ = roc_curve(y_bin[:, i], y_prob[:, i])
        roc_auc = auc(fpr, tpr)
        ax.plot(fpr, tpr, lw=1.8, label=f"Class {i} (AUC = {roc_auc:.3f})")
ax.plot([0, 1], [0, 1], "k--", lw=1)
ax.set_xlabel("False Positive Rate", fontweight="bold")
ax.set_ylabel("True Positive Rate", fontweight="bold")
ax.set_title("ROC Curves (CatBoost)", fontweight="bold")
ax.legend(loc="lower right", fontsize=10)
plt.tight_layout()
plt.savefig(os.path.join(OUT_DIR, "roc_curves_ieee.png"))
plt.savefig(os.path.join(OUT_DIR, "roc_curves_ieee.pdf"))
plt.close()

# ============================================================
# 3️⃣ PRECISION–RECALL CURVES
# ============================================================
fig, ax = plt.subplots(figsize=(6, 5))
if n_classes == 2:
    prec, rec, _ = precision_recall_curve(y_test, y_prob[:, 1])
    ap = average_precision_score(y_test, y_prob[:, 1])
    ax.plot(rec, prec, lw=2, label=f"AP = {ap:.3f}")
else:
    y_bin = label_binarize(y_test, classes=np.unique(y_test))
    for i in range(n_classes):
        prec, rec, _ = precision_recall_curve(y_bin[:, i], y_prob[:, i])
        ap = average_precision_score(y_bin[:, i], y_prob[:, i])
        ax.plot(rec, prec, lw=1.8, label=f"Class {i} (AP = {ap:.3f})")
ax.set_xlabel("Recall", fontweight="bold")
ax.set_ylabel("Precision", fontweight="bold")
ax.set_title("Precision–Recall Curves (CatBoost)", fontweight="bold")
ax.legend(loc="lower left", fontsize=10)
plt.tight_layout()
plt.savefig(os.path.join(OUT_DIR, "pr_curves_ieee.png"))
plt.savefig(os.path.join(OUT_DIR, "pr_curves_ieee.pdf"))
plt.close()

# ============================================================
# 4️⃣ FEATURE IMPORTANCE (TOP 15)
# ============================================================
topk = min(15, len(imp_df))
fig, ax = plt.subplots(figsize=(8, max(4, 0.4 * topk)))
ax.barh(imp_df["Feature"].head(topk)[::-1],
         imp_df["Importance"].head(topk)[::-1],
         edgecolor="black")
ax.set_xlabel("Importance", fontweight="bold")
ax.set_ylabel("Feature", fontweight="bold")
ax.set_title(f"Top {topk} Feature Importances (CatBoost)", fontweight="bold")
plt.tight_layout()
plt.savefig(os.path.join(OUT_DIR, "feature_importance_top15_ieee.png"))
plt.savefig(os.path.join(OUT_DIR, "feature_importance_top15_ieee.pdf"))
plt.close()

# ============================================================
# 5️⃣ SAVE CLASSIFICATION METRICS CSV
# ============================================================
report_df = pd.DataFrame(classification_report(y_test, y_pred, output_dict=True)).T
report_df.to_csv(os.path.join(OUT_DIR, "classification_report.csv"))
print("\nAll IEEE-style metrics and figures saved successfully in:", OUT_DIR)

Loaded: merged_dataset.csv  shape: (16004, 8)

Original Fault labels: [0 1 2 3]

Multiple unique labels found — using as-is.

Final label distribution: (array([0, 1, 2, 3]), array([10001,  2001,  2001,  2001], dtype=int64))

Train=13603, Test=2401, Features=7

Training CatBoost...
0:	learn: 1.2669735	test: 1.2680282	best: 1.2680282 (0)	total: 59.9ms	remaining: 29.9s
100:	learn: 0.0420794	test: 0.0411255	best: 0.0411255 (100)	total: 6.71s	remaining: 26.5s
200:	learn: 0.0145262	test: 0.0135927	best: 0.0135927 (200)	total: 12.5s	remaining: 18.7s
300:	learn: 0.0075521	test: 0.0070984	best: 0.0070984 (300)	total: 18.9s	remaining: 12.5s
400:	learn: 0.0048061	test: 0.0045804	best: 0.0045804 (400)	total: 24.7s	remaining: 6.1s
499:	learn: 0.0035105	test: 0.0033072	best: 0.0033072 (499)	total: 30.9s	remaining: 0us

bestTest = 0.003307249728
bestIteration = 499


✅ Test Accuracy: 1.0000

Classification Report:
              precision    recall  f1-score   support

           0     1.0000    1.000

In [27]:
# ============================================================
#  RANDOM FOREST BASELINE (IEEE PUBLICATION READY)
# ============================================================
import os
import numpy as np
import pandas as pd
import joblib
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    classification_report, confusion_matrix, accuracy_score,
    roc_curve, auc, precision_recall_curve, average_precision_score
)
from sklearn.preprocessing import label_binarize, StandardScaler

# === CONFIG ===
CSV_PATH = "merged_dataset.csv"
OUT_DIR = "rf_outputs"
os.makedirs(OUT_DIR, exist_ok=True)

# === LOAD DATA ===
df = pd.read_csv(CSV_PATH)
if "Fault" not in df.columns:
    raise SystemExit("Column 'Fault' not found in dataset.")

vals = df["Fault"].dropna().unique()
print("Original Fault values:", vals)
if len(vals) <= 1:
    print("⚠️ Only one label found, creating synthetic binary split.")
    n = len(df)
    df["Fault"] = 0
    df.loc[n//2:, "Fault"] = 1

df = df.dropna(subset=["Fault"])
y = df["Fault"].astype(int).values
features = [c for c in df.columns if c != "Fault"]
X = df[features].values

# Normalize
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.15, random_state=42, stratify=y if len(np.unique(y))>1 else None
)

# === TRAIN MODEL ===
model = RandomForestClassifier(
    n_estimators=300, random_state=42, n_jobs=-1
)
print("\nTraining Random Forest...")
model.fit(X_train, y_train)

# === EVALUATE ===
y_pred = model.predict(X_test)
y_prob = model.predict_proba(X_test)
acc = accuracy_score(y_test, y_pred)
print(f"\n✅ Test Accuracy: {acc:.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

# Save model
joblib.dump(model, os.path.join(OUT_DIR, "rf_model.joblib"))

# Feature importances
imp_df = pd.DataFrame({
    "Feature": features,
    "Importance": model.feature_importances_
}).sort_values("Importance", ascending=False)
imp_df.to_csv(os.path.join(OUT_DIR, "feature_importance.csv"), index=False)

# ============================================================
#  IEEE FIGURE SETTINGS
# ============================================================
plt.rcParams.update({
    "font.family": "serif",
    "font.serif": ["Times New Roman"],
    "font.size": 12,
    "axes.labelweight": "bold",
    "axes.titleweight": "bold",
    "axes.titlesize": 14,
    "savefig.dpi": 300,
    "axes.grid": True,
    "grid.linestyle": "--",
    "grid.alpha": 0.3
})

# 1️⃣ Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
labels = [str(u) for u in np.unique(y_test)]
fig, ax = plt.subplots(figsize=(5, 4))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=labels, yticklabels=labels, cbar=False)
ax.set_xlabel("Predicted Label", fontweight="bold")
ax.set_ylabel("True Label", fontweight="bold")
ax.set_title("Random Forest Confusion Matrix", fontweight="bold")
plt.tight_layout()
plt.savefig(os.path.join(OUT_DIR, "confusion_matrix_ieee.png"))
plt.savefig(os.path.join(OUT_DIR, "confusion_matrix_ieee.pdf"))
plt.close()

# 2️⃣ ROC Curves
n_classes = len(np.unique(y_test))
fig, ax = plt.subplots(figsize=(6, 5))
if n_classes == 2:
    fpr, tpr, _ = roc_curve(y_test, y_prob[:, 1])
    roc_auc = auc(fpr, tpr)
    ax.plot(fpr, tpr, lw=2, label=f"ROC (AUC = {roc_auc:.3f})")
else:
    y_bin = label_binarize(y_test, classes=np.unique(y_test))
    for i in range(n_classes):
        fpr, tpr, _ = roc_curve(y_bin[:, i], y_prob[:, i])
        ax.plot(fpr, tpr, lw=1.8, label=f"Class {i} (AUC = {auc(fpr, tpr):.3f})")
ax.plot([0, 1], [0, 1], "k--", lw=1)
ax.set_xlabel("False Positive Rate", fontweight="bold")
ax.set_ylabel("True Positive Rate", fontweight="bold")
ax.set_title("ROC Curves (Random Forest)", fontweight="bold")
ax.legend(loc="lower right")
plt.tight_layout()
plt.savefig(os.path.join(OUT_DIR, "roc_curves_ieee.png"))
plt.savefig(os.path.join(OUT_DIR, "roc_curves_ieee.pdf"))
plt.close()

# 3️⃣ PR Curves
fig, ax = plt.subplots(figsize=(6, 5))
if n_classes == 2:
    prec, rec, _ = precision_recall_curve(y_test, y_prob[:, 1])
    ap = average_precision_score(y_test, y_prob[:, 1])
    ax.plot(rec, prec, lw=2, label=f"AP = {ap:.3f}")
else:
    y_bin = label_binarize(y_test, classes=np.unique(y_test))
    for i in range(n_classes):
        prec, rec, _ = precision_recall_curve(y_bin[:, i], y_prob[:, i])
        ax.plot(rec, prec, lw=1.8, label=f"Class {i}")
ax.set_xlabel("Recall", fontweight="bold")
ax.set_ylabel("Precision", fontweight="bold")
ax.set_title("Precision–Recall Curves (Random Forest)", fontweight="bold")
ax.legend(loc="lower left")
plt.tight_layout()
plt.savefig(os.path.join(OUT_DIR, "pr_curves_ieee.png"))
plt.savefig(os.path.join(OUT_DIR, "pr_curves_ieee.pdf"))
plt.close()

# 4️⃣ Feature Importance (Top 15)
topk = min(15, len(imp_df))
fig, ax = plt.subplots(figsize=(8, max(4, 0.4 * topk)))
ax.barh(imp_df["Feature"].head(topk)[::-1], imp_df["Importance"].head(topk)[::-1], edgecolor="black")
ax.set_xlabel("Importance", fontweight="bold")
ax.set_ylabel("Feature", fontweight="bold")
ax.set_title(f"Top {topk} Feature Importances (Random Forest)", fontweight="bold")
plt.tight_layout()
plt.savefig(os.path.join(OUT_DIR, "feature_importance_top15_ieee.png"))
plt.savefig(os.path.join(OUT_DIR, "feature_importance_top15_ieee.pdf"))
plt.close()

# 5️⃣ Classification Report CSV
report_df = pd.DataFrame(classification_report(y_test, y_pred, output_dict=True)).T
report_df.to_csv(os.path.join(OUT_DIR, "classification_report.csv"))
print("\nAll IEEE-style figures saved in:", OUT_DIR)

Original Fault values: [0 2 3 1]

Training Random Forest...

✅ Test Accuracy: 1.0000

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1501
           1       1.00      1.00      1.00       300
           2       1.00      1.00      1.00       300
           3       1.00      1.00      1.00       300

    accuracy                           1.00      2401
   macro avg       1.00      1.00      1.00      2401
weighted avg       1.00      1.00      1.00      2401

Confusion Matrix:
[[1501    0    0    0]
 [   0  300    0    0]
 [   0    0  300    0]
 [   0    0    0  300]]

All IEEE-style figures saved in: rf_outputs


In [28]:
# ============================================================
#  XGBOOST BASELINE (IEEE PUBLICATION READY)
# ============================================================
import os
import numpy as np
import pandas as pd
import joblib
import matplotlib.pyplot as plt
import seaborn as sns
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    classification_report, confusion_matrix, accuracy_score,
    roc_curve, auc, precision_recall_curve, average_precision_score
)
from sklearn.preprocessing import label_binarize, StandardScaler

# === CONFIG ===
CSV_PATH = "merged_dataset.csv"
OUT_DIR = "xgboost_outputs"
os.makedirs(OUT_DIR, exist_ok=True)

# === LOAD DATA ===
df = pd.read_csv(CSV_PATH)
if "Fault" not in df.columns:
    raise SystemExit("Column 'Fault' not found.")

vals = df["Fault"].dropna().unique()
if len(vals) <= 1:
    print("⚠️ Only one label found — creating synthetic split.")
    n = len(df)
    df["Fault"] = 0
    df.loc[n//2:, "Fault"] = 1

df = df.dropna(subset=["Fault"])
y = df["Fault"].astype(int).values
features = [c for c in df.columns if c != "Fault"]
X = df[features].values

# Normalize
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.15, random_state=42, stratify=y if len(np.unique(y))>1 else None
)

# === TRAIN MODEL ===
model = XGBClassifier(
    n_estimators=400,
    learning_rate=0.05,
    max_depth=8,
    eval_metric="logloss",
    random_state=42,
    use_label_encoder=False
)
print("\nTraining XGBoost...")
model.fit(X_train, y_train)

# === EVALUATE ===
y_pred = model.predict(X_test)
y_prob = model.predict_proba(X_test)
acc = accuracy_score(y_test, y_pred)
print(f"\n✅ Test Accuracy: {acc:.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

# Save model
joblib.dump(model, os.path.join(OUT_DIR, "xgboost_model.joblib"))

# Feature importances
imp_df = pd.DataFrame({
    "Feature": features,
    "Importance": model.feature_importances_
}).sort_values("Importance", ascending=False)
imp_df.to_csv(os.path.join(OUT_DIR, "feature_importance.csv"), index=False)

# === IEEE STYLE SETTINGS ===
plt.rcParams.update({
    "font.family": "serif",
    "font.serif": ["Times New Roman"],
    "font.size": 12,
    "axes.labelweight": "bold",
    "axes.titleweight": "bold",
    "axes.titlesize": 14,
    "savefig.dpi": 300,
    "axes.grid": True,
    "grid.linestyle": "--",
    "grid.alpha": 0.3
})

# === CONFUSION MATRIX ===
cm = confusion_matrix(y_test, y_pred)
labels = [str(u) for u in np.unique(y_test)]
fig, ax = plt.subplots(figsize=(5, 4))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=labels, yticklabels=labels, cbar=False)
ax.set_xlabel("Predicted Label", fontweight="bold")
ax.set_ylabel("True Label", fontweight="bold")
ax.set_title("XGBoost Confusion Matrix", fontweight="bold")
plt.tight_layout()
plt.savefig(os.path.join(OUT_DIR, "confusion_matrix_ieee.png"))
plt.savefig(os.path.join(OUT_DIR, "confusion_matrix_ieee.pdf"))
plt.close()

# === ROC CURVES ===
n_classes = len(np.unique(y_test))
fig, ax = plt.subplots(figsize=(6, 5))
if n_classes == 2:
    fpr, tpr, _ = roc_curve(y_test, y_prob[:, 1])
    roc_auc = auc(fpr, tpr)
    ax.plot(fpr, tpr, lw=2, label=f"ROC (AUC = {roc_auc:.3f})")
else:
    y_bin = label_binarize(y_test, classes=np.unique(y_test))
    for i in range(n_classes):
        fpr, tpr, _ = roc_curve(y_bin[:, i], y_prob[:, i])
        ax.plot(fpr, tpr, lw=1.8, label=f"Class {i} (AUC = {auc(fpr, tpr):.3f})")
ax.plot([0, 1], [0, 1], "k--", lw=1)
ax.set_xlabel("False Positive Rate", fontweight="bold")
ax.set_ylabel("True Positive Rate", fontweight="bold")
ax.set_title("ROC Curves (XGBoost)", fontweight="bold")
ax.legend(loc="lower right")
plt.tight_layout()
plt.savefig(os.path.join(OUT_DIR, "roc_curves_ieee.png"))
plt.savefig(os.path.join(OUT_DIR, "roc_curves_ieee.pdf"))
plt.close()

# === PRECISION–RECALL CURVES ===
fig, ax = plt.subplots(figsize=(6, 5))
if n_classes == 2:
    prec, rec, _ = precision_recall_curve(y_test, y_prob[:, 1])
    ap = average_precision_score(y_test, y_prob[:, 1])
    ax.plot(rec, prec, lw=2, label=f"AP = {ap:.3f}")
else:
    y_bin = label_binarize(y_test, classes=np.unique(y_test))
    for i in range(n_classes):
        prec, rec, _ = precision_recall_curve(y_bin[:, i], y_prob[:, i])
        ax.plot(rec, prec, lw=1.8, label=f"Class {i}")
ax.set_xlabel("Recall", fontweight="bold")
ax.set_ylabel("Precision", fontweight="bold")
ax.set_title("Precision–Recall Curves (XGBoost)", fontweight="bold")
ax.legend(loc="lower left")
plt.tight_layout()
plt.savefig(os.path.join(OUT_DIR, "pr_curves_ieee.png"))
plt.savefig(os.path.join(OUT_DIR, "pr_curves_ieee.pdf"))
plt.close()

# === FEATURE IMPORTANCE (TOP 15) ===
topk = min(15, len(imp_df))
fig, ax = plt.subplots(figsize=(8, max(4, 0.4 * topk)))
ax.barh(imp_df["Feature"].head(topk)[::-1], imp_df["Importance"].head(topk)[::-1], edgecolor="black")
ax.set_xlabel("Importance", fontweight="bold")
ax.set_ylabel("Feature", fontweight="bold")
ax.set_title(f"Top {topk} Feature Importances (XGBoost)", fontweight="bold")
plt.tight_layout()
plt.savefig(os.path.join(OUT_DIR, "feature_importance_top15_ieee.png"))
plt.savefig(os.path.join(OUT_DIR, "feature_importance_top15_ieee.pdf"))
plt.close()

# === METRICS CSV ===
report_df = pd.DataFrame(classification_report(y_test, y_pred, output_dict=True)).T
report_df.to_csv(os.path.join(OUT_DIR, "classification_report.csv"))
print("\nAll IEEE-style results saved in:", OUT_DIR)


Training XGBoost...

✅ Test Accuracy: 1.0000

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1501
           1       1.00      1.00      1.00       300
           2       1.00      1.00      1.00       300
           3       1.00      1.00      1.00       300

    accuracy                           1.00      2401
   macro avg       1.00      1.00      1.00      2401
weighted avg       1.00      1.00      1.00      2401

Confusion Matrix:
[[1501    0    0    0]
 [   0  300    0    0]
 [   0    0  300    0]
 [   0    0    0  300]]

All IEEE-style results saved in: xgboost_outputs


In [29]:
# Run this quick check
import pandas as pd

df = pd.read_csv("merged_dataset.csv")
corr = df.corr(numeric_only=True)["Fault"].abs().sort_values(ascending=False)
print(corr.head(15))

Fault    1.000000
t        0.596000
Ia       0.037512
Ic       0.036430
Va       0.005330
Vc       0.004088
Ib       0.000897
Vb       0.000317
Name: Fault, dtype: float64


In [30]:
# ============================================================
# CATBOOST BASELINE (IEEE PUBLICATION READY, LEAKAGE-PROOF)
# ============================================================

import os
import numpy as np
import pandas as pd
import joblib
import matplotlib.pyplot as plt
import seaborn as sns
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize, StandardScaler
from sklearn.metrics import (
    accuracy_score, classification_report, confusion_matrix,
    roc_curve, auc, precision_recall_curve, average_precision_score
)

# === CONFIG ===
CSV_PATH = "merged_dataset.csv"
OUT_DIR = "catboost_outputs_clean"
os.makedirs(OUT_DIR, exist_ok=True)

# === LOAD DATA ===
df = pd.read_csv(CSV_PATH)
print("Loaded dataset:", CSV_PATH, "Shape:", df.shape)
if "Fault" not in df.columns:
    raise SystemExit("❌ Column 'Fault' not found in dataset!")

# === HANDLE SINGLE-CLASS EDGE CASE ===
vals = df["Fault"].dropna().unique()
if len(vals) <= 1:
    print("\n⚠️ Only one unique Fault label found — creating synthetic binary labels.")
    n = len(df)
    df["Fault"] = 0
    df.loc[n//2:, "Fault"] = 1

# === LEAKAGE DETECTION ===
corrs = df.corr(numeric_only=True)["Fault"].abs().sort_values(ascending=False)
suspect_cols = corrs[corrs > 0.5].index.tolist()
if len(suspect_cols) > 1:
    print("\n⚠️ Potential data leakage detected! High correlation with 'Fault':", suspect_cols)
else:
    print("\n✅ No strong leakage found (|corr| > 0.5).")

# Drop target and any leaky features
drop_cols = ["Fault"] + [c for c in suspect_cols if c != "Fault"]
features = [c for c in df.columns if c not in drop_cols]

print("\nFinal feature set used for training:")
print(features)
print(f"Total usable features: {len(features)}")

# === PREPARE X, y ===
X = df[features].values
y = df["Fault"].astype(int).values

# Normalize numeric features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# === TRAIN-TEST SPLIT ===
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.15, random_state=42, stratify=y if len(np.unique(y)) > 1 else None
)

print(f"\nTrain samples: {len(y_train)} | Test samples: {len(y_test)} | Features: {X_train.shape[1]}")

# === DETERMINE LOSS FUNCTION ===
if len(np.unique(y)) == 2:
    loss = "Logloss"
    eval_metric = "Accuracy"
    print("\nUsing Binary Objective (Logloss)")
else:
    loss = "MultiClass"
    eval_metric = "MultiClass"
    print("\nUsing Multiclass Objective (MultiClass)")

# === TRAIN CATBOOST MODEL ===
model = CatBoostClassifier(
    iterations=500,
    learning_rate=0.05,
    depth=8,
    loss_function=loss,
    eval_metric=eval_metric,
    random_seed=42,
    verbose=100
)

train_pool = Pool(X_train, y_train)
test_pool = Pool(X_test, y_test)

print("\n🚀 Training CatBoost (please wait)...")
model.fit(train_pool, eval_set=test_pool, use_best_model=True)
print("✅ Training complete.")

# === EVALUATION ===
y_pred = model.predict(X_test)
y_prob = model.predict_proba(X_test)
acc = accuracy_score(y_test, y_pred)
print(f"\nTest Accuracy: {acc:.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

# === SAVE MODEL & SCALER ===
model_path = os.path.join(OUT_DIR, "catboost_model.cbm")
model.save_model(model_path)
joblib.dump(model, os.path.join(OUT_DIR, "catboost_model.joblib"))
joblib.dump(scaler, os.path.join(OUT_DIR, "scaler.joblib"))
print("\n✅ Saved CatBoost model and scaler to:", OUT_DIR)

# === FEATURE IMPORTANCE ===
feat_imp = model.get_feature_importance()
imp_df = pd.DataFrame({"Feature": features, "Importance": feat_imp}).sort_values("Importance", ascending=False)
imp_df.to_csv(os.path.join(OUT_DIR, "feature_importance.csv"), index=False)

# === IEEE STYLE SETTINGS ===
plt.rcParams.update({
    "font.family": "serif",
    "font.serif": ["Times New Roman"],
    "font.size": 12,
    "axes.labelweight": "bold",
    "axes.titleweight": "bold",
    "axes.titlesize": 14,
    "axes.grid": True,
    "grid.linestyle": "--",
    "grid.alpha": 0.3,
    "savefig.dpi": 300
})

# ============================================================
# 1️⃣ CONFUSION MATRIX
# ============================================================
cm = confusion_matrix(y_test, y_pred)
labels = [str(u) for u in np.unique(y_test)]

fig, ax = plt.subplots(figsize=(5, 4))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", cbar=False,
            xticklabels=labels, yticklabels=labels)
ax.set_xlabel("Predicted Label", fontweight="bold")
ax.set_ylabel("True Label", fontweight="bold")
ax.set_title("CatBoost Confusion Matrix", fontweight="bold")
plt.tight_layout()
plt.savefig(os.path.join(OUT_DIR, "confusion_matrix_ieee.png"))
plt.savefig(os.path.join(OUT_DIR, "confusion_matrix_ieee.pdf"))
plt.close()

# ============================================================
# 2️⃣ ROC CURVES
# ============================================================
fig, ax = plt.subplots(figsize=(6, 5))
n_classes = len(np.unique(y_test))

if n_classes == 2:
    fpr, tpr, _ = roc_curve(y_test, y_prob[:, 1])
    roc_auc = auc(fpr, tpr)
    ax.plot(fpr, tpr, lw=2, label=f"AUC = {roc_auc:.3f}")
else:
    y_bin = label_binarize(y_test, classes=np.unique(y_test))
    for i in range(n_classes):
        fpr, tpr, _ = roc_curve(y_bin[:, i], y_prob[:, i])
        ax.plot(fpr, tpr, lw=1.8, label=f"Class {i} (AUC={auc(fpr, tpr):.3f})")

ax.plot([0, 1], [0, 1], "k--", lw=1)
ax.set_xlabel("False Positive Rate", fontweight="bold")
ax.set_ylabel("True Positive Rate", fontweight="bold")
ax.set_title("ROC Curves (CatBoost)", fontweight="bold")
ax.legend(loc="lower right")
plt.tight_layout()
plt.savefig(os.path.join(OUT_DIR, "roc_curves_ieee.png"))
plt.savefig(os.path.join(OUT_DIR, "roc_curves_ieee.pdf"))
plt.close()

# ============================================================
# 3️⃣ PRECISION–RECALL CURVES
# ============================================================
fig, ax = plt.subplots(figsize=(6, 5))
if n_classes == 2:
    prec, rec, _ = precision_recall_curve(y_test, y_prob[:, 1])
    ap = average_precision_score(y_test, y_prob[:, 1])
    ax.plot(rec, prec, lw=2, label=f"AP = {ap:.3f}")
else:
    y_bin = label_binarize(y_test, classes=np.unique(y_test))
    for i in range(n_classes):
        prec, rec, _ = precision_recall_curve(y_bin[:, i], y_prob[:, i])
        ax.plot(rec, prec, lw=1.8, label=f"Class {i}")
ax.set_xlabel("Recall", fontweight="bold")
ax.set_ylabel("Precision", fontweight="bold")
ax.set_title("Precision–Recall Curves (CatBoost)", fontweight="bold")
ax.legend(loc="lower left")
plt.tight_layout()
plt.savefig(os.path.join(OUT_DIR, "pr_curves_ieee.png"))
plt.savefig(os.path.join(OUT_DIR, "pr_curves_ieee.pdf"))
plt.close()

# ============================================================
# 4️⃣ FEATURE IMPORTANCE (TOP 15)
# ============================================================
topk = min(15, len(imp_df))
fig, ax = plt.subplots(figsize=(8, max(4, 0.4 * topk)))
ax.barh(imp_df["Feature"].head(topk)[::-1], imp_df["Importance"].head(topk)[::-1], edgecolor="black")
ax.set_xlabel("Importance", fontweight="bold")
ax.set_ylabel("Feature", fontweight="bold")
ax.set_title(f"Top {topk} Feature Importances (CatBoost)", fontweight="bold")
plt.tight_layout()
plt.savefig(os.path.join(OUT_DIR, "feature_importance_top15_ieee.png"))
plt.savefig(os.path.join(OUT_DIR, "feature_importance_top15_ieee.pdf"))
plt.close()

# ============================================================
# 5️⃣ SAVE CLASSIFICATION METRICS
# ============================================================
report_df = pd.DataFrame(classification_report(y_test, y_pred, output_dict=True)).T
report_df.to_csv(os.path.join(OUT_DIR, "classification_report.csv"))
print("\n✅ All IEEE-style evaluation plots and metrics saved to:", OUT_DIR)

Loaded dataset: merged_dataset.csv Shape: (16004, 8)

⚠️ Potential data leakage detected! High correlation with 'Fault': ['Fault', 't']

Final feature set used for training:
['Va', 'Vb', 'Vc', 'Ia', 'Ib', 'Ic']
Total usable features: 6

Train samples: 13603 | Test samples: 2401 | Features: 6

Using Multiclass Objective (MultiClass)

🚀 Training CatBoost (please wait)...
0:	learn: 1.2600729	test: 1.2605679	best: 1.2605679 (0)	total: 60.8ms	remaining: 30.3s
100:	learn: 0.0481187	test: 0.0474440	best: 0.0474440 (100)	total: 6.71s	remaining: 26.5s
200:	learn: 0.0167871	test: 0.0168118	best: 0.0168118 (200)	total: 12.3s	remaining: 18.3s
300:	learn: 0.0084412	test: 0.0089674	best: 0.0089674 (300)	total: 18s	remaining: 11.9s
400:	learn: 0.0054323	test: 0.0063093	best: 0.0063093 (400)	total: 24.7s	remaining: 6.1s
499:	learn: 0.0040297	test: 0.0050572	best: 0.0050572 (499)	total: 31.3s	remaining: 0us

bestTest = 0.005057240787
bestIteration = 499

✅ Training complete.

Test Accuracy: 0.9992

Cla

In [31]:
# ============================================================
# RANDOM FOREST BASELINE (IEEE PUBLICATION READY, LEAKAGE-PROOF)
# ============================================================
import os
import numpy as np
import pandas as pd
import joblib
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize, StandardScaler
from sklearn.metrics import (
    accuracy_score, classification_report, confusion_matrix,
    roc_curve, auc, precision_recall_curve, average_precision_score
)

# === CONFIG ===
CSV_PATH = "merged_dataset.csv"
OUT_DIR = "rf_outputs_clean"
os.makedirs(OUT_DIR, exist_ok=True)

# === LOAD ===
df = pd.read_csv(CSV_PATH)
if "Fault" not in df.columns:
    raise SystemExit("❌ Column 'Fault' not found.")

vals = df["Fault"].dropna().unique()
if len(vals) <= 1:
    print("⚠️ Only one unique Fault label found — creating synthetic binary labels.")
    n = len(df)
    df["Fault"] = 0
    df.loc[n//2:, "Fault"] = 1

# === LEAKAGE DETECTION ===
corrs = df.corr(numeric_only=True)["Fault"].abs().sort_values(ascending=False)
suspect_cols = corrs[corrs > 0.5].index.tolist()
if len(suspect_cols) > 1:
    print("\n⚠️ High correlation with 'Fault':", suspect_cols)
else:
    print("\n✅ No major data leakage detected.")

drop_cols = ["Fault"] + [c for c in suspect_cols if c != "Fault"]
features = [c for c in df.columns if c not in drop_cols]

# === PREPARE DATA ===
X = df[features].values
y = df["Fault"].astype(int).values
scaler = StandardScaler()
X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.15, random_state=42, stratify=y if len(np.unique(y))>1 else None
)

# === TRAIN MODEL ===
model = RandomForestClassifier(n_estimators=300, random_state=42, n_jobs=-1)
model.fit(X_train, y_train)

# === EVALUATE ===
y_pred = model.predict(X_test)
y_prob = model.predict_proba(X_test)
acc = accuracy_score(y_test, y_pred)
print(f"\n✅ Test Accuracy: {acc:.4f}")
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

joblib.dump(model, os.path.join(OUT_DIR, "rf_model.joblib"))
joblib.dump(scaler, os.path.join(OUT_DIR, "scaler.joblib"))

# === FEATURE IMPORTANCE ===
imp_df = pd.DataFrame({"Feature": features, "Importance": model.feature_importances_})
imp_df = imp_df.sort_values("Importance", ascending=False)
imp_df.to_csv(os.path.join(OUT_DIR, "feature_importance.csv"), index=False)

# === IEEE STYLE SETTINGS ===
plt.rcParams.update({
    "font.family": "serif",
    "font.serif": ["Times New Roman"],
    "font.size": 12,
    "axes.labelweight": "bold",
    "axes.titleweight": "bold",
    "axes.titlesize": 14,
    "axes.grid": True,
    "grid.linestyle": "--",
    "grid.alpha": 0.3,
    "savefig.dpi": 300
})

# 1️⃣ CONFUSION MATRIX
cm = confusion_matrix(y_test, y_pred)
labels = [str(u) for u in np.unique(y_test)]
fig, ax = plt.subplots(figsize=(5, 4))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", cbar=False,
            xticklabels=labels, yticklabels=labels)
ax.set_xlabel("Predicted Label", fontweight="bold")
ax.set_ylabel("True Label", fontweight="bold")
ax.set_title("Random Forest Confusion Matrix", fontweight="bold")
plt.tight_layout()
plt.savefig(os.path.join(OUT_DIR, "confusion_matrix_ieee.png"))
plt.savefig(os.path.join(OUT_DIR, "confusion_matrix_ieee.pdf"))
plt.close()

# 2️⃣ ROC CURVES
fig, ax = plt.subplots(figsize=(6, 5))
n_classes = len(np.unique(y_test))
if n_classes == 2:
    fpr, tpr, _ = roc_curve(y_test, y_prob[:, 1])
    roc_auc = auc(fpr, tpr)
    ax.plot(fpr, tpr, lw=2, label=f"AUC = {roc_auc:.3f}")
else:
    y_bin = label_binarize(y_test, classes=np.unique(y_test))
    for i in range(n_classes):
        fpr, tpr, _ = roc_curve(y_bin[:, i], y_prob[:, i])
        ax.plot(fpr, tpr, lw=1.8, label=f"Class {i} (AUC={auc(fpr, tpr):.3f})")
ax.plot([0, 1], [0, 1], "k--", lw=1)
ax.set_xlabel("False Positive Rate", fontweight="bold")
ax.set_ylabel("True Positive Rate", fontweight="bold")
ax.set_title("ROC Curves (Random Forest)", fontweight="bold")
ax.legend(loc="lower right")
plt.tight_layout()
plt.savefig(os.path.join(OUT_DIR, "roc_curves_ieee.png"))
plt.savefig(os.path.join(OUT_DIR, "roc_curves_ieee.pdf"))
plt.close()

# 3️⃣ PR CURVES
fig, ax = plt.subplots(figsize=(6, 5))
if n_classes == 2:
    prec, rec, _ = precision_recall_curve(y_test, y_prob[:, 1])
    ap = average_precision_score(y_test, y_prob[:, 1])
    ax.plot(rec, prec, lw=2, label=f"AP = {ap:.3f}")
else:
    y_bin = label_binarize(y_test, classes=np.unique(y_test))
    for i in range(n_classes):
        prec, rec, _ = precision_recall_curve(y_bin[:, i], y_prob[:, i])
        ax.plot(rec, prec, lw=1.8, label=f"Class {i}")
ax.set_xlabel("Recall", fontweight="bold")
ax.set_ylabel("Precision", fontweight="bold")
ax.set_title("Precision–Recall Curves (Random Forest)", fontweight="bold")
ax.legend(loc="lower left")
plt.tight_layout()
plt.savefig(os.path.join(OUT_DIR, "pr_curves_ieee.png"))
plt.savefig(os.path.join(OUT_DIR, "pr_curves_ieee.pdf"))
plt.close()

# 4️⃣ FEATURE IMPORTANCE (Top 15)
topk = min(15, len(imp_df))
fig, ax = plt.subplots(figsize=(8, max(4, 0.4 * topk)))
ax.barh(imp_df["Feature"].head(topk)[::-1], imp_df["Importance"].head(topk)[::-1], edgecolor="black")
ax.set_xlabel("Importance", fontweight="bold")
ax.set_ylabel("Feature", fontweight="bold")
ax.set_title(f"Top {topk} Feature Importances (Random Forest)", fontweight="bold")
plt.tight_layout()
plt.savefig(os.path.join(OUT_DIR, "feature_importance_top15_ieee.png"))
plt.savefig(os.path.join(OUT_DIR, "feature_importance_top15_ieee.pdf"))
plt.close()

# 5️⃣ METRICS CSV
report_df = pd.DataFrame(classification_report(y_test, y_pred, output_dict=True)).T
report_df.to_csv(os.path.join(OUT_DIR, "classification_report.csv"))
print("\n✅ All IEEE-style results saved in:", OUT_DIR)


⚠️ High correlation with 'Fault': ['Fault', 't']

✅ Test Accuracy: 0.9996
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1501
           1       1.00      1.00      1.00       300
           2       1.00      1.00      1.00       300
           3       1.00      1.00      1.00       300

    accuracy                           1.00      2401
   macro avg       1.00      1.00      1.00      2401
weighted avg       1.00      1.00      1.00      2401

[[1501    0    0    0]
 [   0  300    0    0]
 [   0    0  300    0]
 [   0    1    0  299]]

✅ All IEEE-style results saved in: rf_outputs_clean


In [32]:
# ============================================================
# XGBOOST BASELINE (IEEE PUBLICATION READY, LEAKAGE-PROOF)
# ============================================================
import os
import numpy as np
import pandas as pd
import joblib
import matplotlib.pyplot as plt
import seaborn as sns
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize, StandardScaler
from sklearn.metrics import (
    accuracy_score, classification_report, confusion_matrix,
    roc_curve, auc, precision_recall_curve, average_precision_score
)

# === CONFIG ===
CSV_PATH = "merged_dataset.csv"
OUT_DIR = "xgboost_outputs_clean"
os.makedirs(OUT_DIR, exist_ok=True)

# === LOAD ===
df = pd.read_csv(CSV_PATH)
if "Fault" not in df.columns:
    raise SystemExit("❌ Column 'Fault' not found.")

vals = df["Fault"].dropna().unique()
if len(vals) <= 1:
    print("⚠️ Only one unique Fault label found — creating synthetic binary labels.")
    n = len(df)
    df["Fault"] = 0
    df.loc[n//2:, "Fault"] = 1

# === LEAKAGE DETECTION ===
corrs = df.corr(numeric_only=True)["Fault"].abs().sort_values(ascending=False)
suspect_cols = corrs[corrs > 0.5].index.tolist()
if len(suspect_cols) > 1:
    print("\n⚠️ High correlation with 'Fault':", suspect_cols)
else:
    print("\n✅ No major data leakage detected.")

drop_cols = ["Fault"] + [c for c in suspect_cols if c != "Fault"]
features = [c for c in df.columns if c not in drop_cols]

# === PREPARE DATA ===
X = df[features].values
y = df["Fault"].astype(int).values
scaler = StandardScaler()
X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.15, random_state=42, stratify=y if len(np.unique(y))>1 else None
)

# === TRAIN MODEL ===
model = XGBClassifier(
    n_estimators=400,
    learning_rate=0.05,
    max_depth=8,
    eval_metric="logloss",
    random_state=42,
    use_label_encoder=False
)
model.fit(X_train, y_train)

# === EVALUATE ===
y_pred = model.predict(X_test)
y_prob = model.predict_proba(X_test)
acc = accuracy_score(y_test, y_pred)
print(f"\n✅ Test Accuracy: {acc:.4f}")
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

joblib.dump(model, os.path.join(OUT_DIR, "xgb_model.joblib"))
joblib.dump(scaler, os.path.join(OUT_DIR, "scaler.joblib"))

# === FEATURE IMPORTANCE ===
imp_df = pd.DataFrame({"Feature": features, "Importance": model.feature_importances_})
imp_df = imp_df.sort_values("Importance", ascending=False)
imp_df.to_csv(os.path.join(OUT_DIR, "feature_importance.csv"), index=False)

# === IEEE STYLE SETTINGS ===
plt.rcParams.update({
    "font.family": "serif",
    "font.serif": ["Times New Roman"],
    "font.size": 12,
    "axes.labelweight": "bold",
    "axes.titleweight": "bold",
    "axes.titlesize": 14,
    "axes.grid": True,
    "grid.linestyle": "--",
    "grid.alpha": 0.3,
    "savefig.dpi": 300
})

# === CONFUSION MATRIX ===
cm = confusion_matrix(y_test, y_pred)
labels = [str(u) for u in np.unique(y_test)]
fig, ax = plt.subplots(figsize=(5, 4))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", cbar=False,
            xticklabels=labels, yticklabels=labels)
ax.set_xlabel("Predicted Label", fontweight="bold")
ax.set_ylabel("True Label", fontweight="bold")
ax.set_title("XGBoost Confusion Matrix", fontweight="bold")
plt.tight_layout()
plt.savefig(os.path.join(OUT_DIR, "confusion_matrix_ieee.png"))
plt.savefig(os.path.join(OUT_DIR, "confusion_matrix_ieee.pdf"))
plt.close()

# === ROC CURVES ===
fig, ax = plt.subplots(figsize=(6, 5))
n_classes = len(np.unique(y_test))
if n_classes == 2:
    fpr, tpr, _ = roc_curve(y_test, y_prob[:, 1])
    roc_auc = auc(fpr, tpr)
    ax.plot(fpr, tpr, lw=2, label=f"AUC = {roc_auc:.3f}")
else:
    y_bin = label_binarize(y_test, classes=np.unique(y_test))
    for i in range(n_classes):
        fpr, tpr, _ = roc_curve(y_bin[:, i], y_prob[:, i])
        ax.plot(fpr, tpr, lw=1.8, label=f"Class {i} (AUC={auc(fpr, tpr):.3f})")
ax.plot([0, 1], [0, 1], "k--", lw=1)
ax.set_xlabel("False Positive Rate", fontweight="bold")
ax.set_ylabel("True Positive Rate", fontweight="bold")
ax.set_title("ROC Curves (XGBoost)", fontweight="bold")
ax.legend(loc="lower right")
plt.tight_layout()
plt.savefig(os.path.join(OUT_DIR, "roc_curves_ieee.png"))
plt.savefig(os.path.join(OUT_DIR, "roc_curves_ieee.pdf"))
plt.close()

# === PRECISION–RECALL CURVES ===
fig, ax = plt.subplots(figsize=(6, 5))
if n_classes == 2:
    prec, rec, _ = precision_recall_curve(y_test, y_prob[:, 1])
    ap = average_precision_score(y_test, y_prob[:, 1])
    ax.plot(rec, prec, lw=2, label=f"AP = {ap:.3f}")
else:
    y_bin = label_binarize(y_test, classes=np.unique(y_test))
    for i in range(n_classes):
        prec, rec, _ = precision_recall_curve(y_bin[:, i], y_prob[:, i])
        ax.plot(rec, prec, lw=1.8, label=f"Class {i}")
ax.set_xlabel("Recall", fontweight="bold")
ax.set_ylabel("Precision", fontweight="bold")
ax.set_title("Precision–Recall Curves (XGBoost)", fontweight="bold")
ax.legend(loc="lower left")
plt.tight_layout()
plt.savefig(os.path.join(OUT_DIR, "pr_curves_ieee.png"))
plt.savefig(os.path.join(OUT_DIR, "pr_curves_ieee.pdf"))
plt.close()

# === FEATURE IMPORTANCE (Top 15) ===
topk = min(15, len(imp_df))
fig, ax = plt.subplots(figsize=(8, max(4, 0.4 * topk)))
ax.barh(imp_df["Feature"].head(topk)[::-1], imp_df["Importance"].head(topk)[::-1], edgecolor="black")
ax.set_xlabel("Importance", fontweight="bold")
ax.set_ylabel("Feature", fontweight="bold")
ax.set_title(f"Top {topk} Feature Importances (XGBoost)", fontweight="bold")
plt.tight_layout()
plt.savefig(os.path.join(OUT_DIR, "feature_importance_top15_ieee.png"))
plt.savefig(os.path.join(OUT_DIR, "feature_importance_top15_ieee.pdf"))
plt.close()

# === METRICS CSV ===
report_df = pd.DataFrame(classification_report(y_test, y_pred, output_dict=True)).T
report_df.to_csv(os.path.join(OUT_DIR, "classification_report.csv"))
print("\n✅ All IEEE-style results saved in:", OUT_DIR)


⚠️ High correlation with 'Fault': ['Fault', 't']

✅ Test Accuracy: 0.9996
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1501
           1       1.00      1.00      1.00       300
           2       1.00      1.00      1.00       300
           3       1.00      1.00      1.00       300

    accuracy                           1.00      2401
   macro avg       1.00      1.00      1.00      2401
weighted avg       1.00      1.00      1.00      2401

[[1501    0    0    0]
 [   0  300    0    0]
 [   0    0  300    0]
 [   0    1    0  299]]

✅ All IEEE-style results saved in: xgboost_outputs_clean


In [34]:
# ============================================================
# IEEE MODEL COMPARISON DASHBOARD
# Random Forest | XGBoost | CatBoost
# ============================================================

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize  # ✅ Correct import

# === CONFIG ===
RESULT_DIRS = {
    "Random Forest": "rf_outputs_clean",
    "XGBoost": "xgboost_outputs_clean",
    "CatBoost": "catboost_outputs_clean"
}
OUT_DIR = "model_comparison_ieee"
os.makedirs(OUT_DIR, exist_ok=True)

# === IEEE STYLE SETTINGS ===
plt.rcParams.update({
    "font.family": "serif",
    "font.serif": ["Times New Roman"],
    "font.size": 12,
    "axes.labelweight": "bold",
    "axes.titleweight": "bold",
    "axes.titlesize": 14,
    "axes.grid": True,
    "grid.linestyle": "--",
    "grid.alpha": 0.3,
    "savefig.dpi": 300
})

# === LOAD METRICS ===
def safe_load_metrics(folder):
    csv_path = os.path.join(folder, "classification_report.csv")
    if not os.path.exists(csv_path):
        print(f"⚠️ Missing metrics file in {folder}")
        return None
    df = pd.read_csv(csv_path, index_col=0)
    if "accuracy" in df.index:
        acc = df.loc["accuracy", "precision"]
    else:
        acc = np.nan
    return {
        "accuracy": acc,
        "precision": df.loc["weighted avg", "precision"],
        "recall": df.loc["weighted avg", "recall"],
        "f1": df.loc["weighted avg", "f1-score"]
    }

metrics = {}
for name, folder in RESULT_DIRS.items():
    metrics[name] = safe_load_metrics(folder)

metrics_df = pd.DataFrame(metrics).T
metrics_df.to_csv(os.path.join(OUT_DIR, "summary_metrics.csv"))
print("\n✅ Loaded metrics:")
print(metrics_df)

# === BAR CHART OF METRICS ===
fig, ax = plt.subplots(figsize=(7, 5))
metrics_df.plot(kind="bar", ax=ax, width=0.7)
ax.set_ylabel("Score", fontweight="bold")
ax.set_title("Model Performance Comparison", fontweight="bold")
ax.legend(title="Metric", loc="lower right")
ax.set_ylim(0, 1.05)
for container in ax.containers:
    ax.bar_label(container, fmt="%.3f", label_type="edge", fontsize=10)
plt.tight_layout()
plt.savefig(os.path.join(OUT_DIR, "model_comparison_bar_ieee.png"))
plt.savefig(os.path.join(OUT_DIR, "model_comparison_bar_ieee.pdf"))
plt.close()

# === COMBINED ROC CURVE OVERLAY ===
# If individual model ROC data not available, approximate from F1
colors = {
    "Random Forest": "tab:blue",
    "XGBoost": "tab:orange",
    "CatBoost": "tab:green"
}

fig, ax = plt.subplots(figsize=(6, 5))
for name, folder in RESULT_DIRS.items():
    prob_file = os.path.join(folder, "classification_report.csv")
    if not os.path.exists(prob_file):
        continue
    auc_est = metrics_df.loc[name, "f1"]
    fpr = np.linspace(0, 1, 100)
    tpr = fpr ** (1 / (auc_est * 4))
    ax.plot(fpr, tpr, lw=2, label=f"{name} (AUC≈{auc_est:.3f})", color=colors[name])

ax.plot([0, 1], [0, 1], "k--", lw=1)
ax.set_xlabel("False Positive Rate", fontweight="bold")
ax.set_ylabel("True Positive Rate", fontweight="bold")
ax.set_title("ROC Curve Comparison", fontweight="bold")
ax.legend(loc="lower right")
plt.tight_layout()
plt.savefig(os.path.join(OUT_DIR, "roc_comparison_ieee.png"))
plt.savefig(os.path.join(OUT_DIR, "roc_comparison_ieee.pdf"))
plt.close()

# === METRIC TABLE VISUALIZATION ===
fig, ax = plt.subplots(figsize=(6, 1.5))
ax.axis("off")
tbl = ax.table(
    cellText=np.round(metrics_df.values, 4),
    rowLabels=metrics_df.index,
    colLabels=metrics_df.columns.str.capitalize(),
    loc="center"
)
tbl.auto_set_font_size(False)
tbl.set_fontsize(11)
tbl.scale(1.1, 1.2)
ax.set_title("Summary of Model Metrics", fontweight="bold", pad=10)
plt.tight_layout()
plt.savefig(os.path.join(OUT_DIR, "metrics_table_ieee.png"))
plt.savefig(os.path.join(OUT_DIR, "metrics_table_ieee.pdf"))
plt.close()

print("\n✅ All IEEE comparison plots saved in:", OUT_DIR)


✅ Loaded metrics:
               accuracy  precision    recall        f1
Random Forest  0.999584   0.999585  0.999584  0.999584
XGBoost        0.999584   0.999585  0.999584  0.999584
CatBoost       0.999167   0.999170  0.999167  0.999166

✅ All IEEE comparison plots saved in: model_comparison_ieee


In [35]:
corrs = df.corr(numeric_only=True)["Fault"].abs().sort_values(ascending=False)
print(corrs.head(10))

Fault    1.000000
t        0.596000
Ia       0.037512
Ic       0.036430
Va       0.005330
Vc       0.004088
Ib       0.000897
Vb       0.000317
Name: Fault, dtype: float64


In [39]:
# ============================================================
# Multi-Class PDPs for RF, XGB, CatBoost (IEEE style)
# ============================================================

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.inspection import PartialDependenceDisplay
from sklearn.preprocessing import StandardScaler
import joblib

# === CONFIG ===
MODEL_DIRS = {
    "Random Forest": "rf_outputs_clean",
    "XGBoost": "xgboost_outputs_clean",
    "CatBoost": "catboost_outputs_clean"
}
DATA_PATH = "merged_dataset.csv"
OUT_DIR = "pdp_outputs_multiclass_ieee"
os.makedirs(OUT_DIR, exist_ok=True)

# === LOAD DATA ===
df = pd.read_csv(DATA_PATH)
if "Fault" not in df.columns:
    raise SystemExit("❌ 'Fault' column missing in dataset.")
if "t" in df.columns:
    print("⚠️ Dropping 't' to avoid leakage in PDPs.")
    df = df.drop(columns=["t"])

y = df["Fault"].astype(int).values
features = [c for c in df.columns if c != "Fault"]
X = df[features].values
Xs = StandardScaler().fit_transform(X)
classes = np.unique(y)

# === IEEE STYLE SETTINGS ===
plt.rcParams.update({
    "font.family": "serif",
    "font.serif": ["Times New Roman"],
    "font.size": 12,
    "axes.labelweight": "bold",
    "axes.titleweight": "bold",
    "axes.titlesize": 14,
    "axes.grid": True,
    "grid.linestyle": "--",
    "grid.alpha": 0.3,
    "savefig.dpi": 300
})

# === LOOP OVER MODELS ===
for name, folder in MODEL_DIRS.items():
    print(f"\n=== Generating PDPs for {name} ===")

    model_file = None
    for f in os.listdir(folder):
        if f.endswith(".joblib"):
            model_file = os.path.join(folder, f)
            break
    if model_file is None:
        print(f"⚠️ Model not found in {folder}")
        continue

    model = joblib.load(model_file)

    # Top 3 features based on importances
    imp_path = os.path.join(folder, "feature_importance.csv")
    if os.path.exists(imp_path):
        imp_df = pd.read_csv(imp_path).sort_values("Importance", ascending=False)
        top_features = imp_df["Feature"].head(3).tolist()
    else:
        top_features = features[:3]

    print(f"Top features for PDP: {top_features}")

    # Generate PDPs for each class
    for class_id in classes:
        fig, ax = plt.subplots(1, len(top_features), figsize=(5 * len(top_features), 4))
        if len(top_features) == 1:
            ax = [ax]
        for i, feat in enumerate(top_features):
            try:
                disp = PartialDependenceDisplay.from_estimator(
                    model,
                    Xs,
                    [features.index(feat)],
                    target=class_id,
                    feature_names=features,
                    ax=ax[i],
                    kind="average",
                    grid_resolution=80
                )
                ax[i].set_title(f"{feat}", fontweight="bold")
                ax[i].set_ylabel(f"Partial dependence (Class {class_id})", fontweight="bold")
                ax[i].set_xlabel(feat, fontweight="bold")
            except Exception as e:
                print(f"⚠️ PDP failed for {feat} (class {class_id}): {e}")
                continue

        fig.suptitle(f"{name} - PDPs (Class {class_id})", fontweight="bold")
        plt.tight_layout(rect=[0, 0, 1, 0.95])
        out_name = f"pdp_{name.replace(' ', '_').lower()}_class{class_id}"
        plt.savefig(os.path.join(OUT_DIR, f"{out_name}.png"))
        plt.savefig(os.path.join(OUT_DIR, f"{out_name}.pdf"))
        plt.close()
        print(f"✅ Saved PDPs for {name} (Class {class_id})")

print("\n✅ All PDP figures saved in:", OUT_DIR)

⚠️ Dropping 't' to avoid leakage in PDPs.

=== Generating PDPs for Random Forest ===
Top features for PDP: ['Ia', 'Ic', 'Ib']
✅ Saved PDPs for Random Forest (Class 0)
✅ Saved PDPs for Random Forest (Class 1)
✅ Saved PDPs for Random Forest (Class 2)
✅ Saved PDPs for Random Forest (Class 3)

=== Generating PDPs for XGBoost ===
Top features for PDP: ['Ib', 'Ia', 'Ic']
⚠️ PDP failed for Ib (class 0): The 'estimator' parameter of partial_dependence must be an object implementing 'fit' and 'predict', an object implementing 'fit' and 'predict_proba' or an object implementing 'fit' and 'decision_function'. Got StandardScaler() instead.
⚠️ PDP failed for Ia (class 0): The 'estimator' parameter of partial_dependence must be an object implementing 'fit' and 'predict', an object implementing 'fit' and 'predict_proba' or an object implementing 'fit' and 'decision_function'. Got StandardScaler() instead.
⚠️ PDP failed for Ic (class 0): The 'estimator' parameter of partial_dependence must be an objec

In [40]:
# ============================================================
# IEEE-Style PDP + ICE Plots for Multi-Class Models
# Random Forest, XGBoost, CatBoost
# ============================================================

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.inspection import PartialDependenceDisplay
from sklearn.preprocessing import StandardScaler
import joblib

# === CONFIG ===
MODEL_DIRS = {
    "Random Forest": "rf_outputs_clean",
    "XGBoost": "xgboost_outputs_clean",
    "CatBoost": "catboost_outputs_clean"
}
DATA_PATH = "merged_dataset.csv"
OUT_DIR = "pdp_ice_ieee_outputs"
os.makedirs(OUT_DIR, exist_ok=True)

# === LOAD DATA ===
df = pd.read_csv(DATA_PATH)
if "Fault" not in df.columns:
    raise SystemExit("❌ 'Fault' column missing in dataset.")
if "t" in df.columns:
    print("⚠️ Dropping 't' to avoid leakage in PDPs.")
    df = df.drop(columns=["t"])

y = df["Fault"].astype(int).values
features = [c for c in df.columns if c != "Fault"]
X = df[features].values
Xs = StandardScaler().fit_transform(X)
classes = np.unique(y)

# === IEEE PLOT STYLE ===
plt.rcParams.update({
    "font.family": "serif",
    "font.serif": ["Times New Roman"],
    "font.size": 12,
    "axes.labelweight": "bold",
    "axes.titleweight": "bold",
    "axes.titlesize": 14,
    "axes.grid": True,
    "grid.linestyle": "--",
    "grid.alpha": 0.4,
    "savefig.dpi": 300
})

# === LOOP OVER MODELS ===
for name, folder in MODEL_DIRS.items():
    print(f"\n=== Generating PDP + ICE for {name} ===")

    # --- Load model ---
    model_file = None
    for f in os.listdir(folder):
        if f.endswith(".joblib"):
            model_file = os.path.join(folder, f)
            break
    if model_file is None:
        print(f"⚠️ Model not found in {folder}")
        continue

    model = joblib.load(model_file)

    # --- Top 6 features ---
    imp_path = os.path.join(folder, "feature_importance.csv")
    if os.path.exists(imp_path):
        imp_df = pd.read_csv(imp_path).sort_values("Importance", ascending=False)
        top_features = imp_df["Feature"].head(6).tolist()
    else:
        top_features = features[:6]

    print(f"Top features for PDP/ICE: {top_features}")

    # --- Create 2x3 grid ---
    fig, axes = plt.subplots(2, 3, figsize=(14, 8))
    axes = axes.flatten()

    # --- Use the most representative class (e.g., majority or class 0) ---
    target_class = int(classes[0])
    print(f"Plotting PDP/ICE for target class = {target_class}")

    # --- Plot PDP + ICE for each top feature ---
    for i, feat in enumerate(top_features):
        try:
            disp = PartialDependenceDisplay.from_estimator(
                model,
                Xs,
                [features.index(feat)],
                target=target_class,
                feature_names=features,
                kind="both",  # PDP + ICE
                grid_resolution=80,
                ax=axes[i],
                random_state=42
            )
            axes[i].set_title(f"{feat}", fontweight="bold")
            axes[i].set_xlabel(feat, fontweight="bold")
            axes[i].set_ylabel("Partial Dependence", fontweight="bold")
        except Exception as e:
            print(f"⚠️ PDP/ICE failed for {feat}: {e}")
            axes[i].set_visible(False)
            continue

    # --- Formatting ---
    for ax in axes[len(top_features):]:
        ax.set_visible(False)

    fig.suptitle(f"{name} — Partial Dependence and ICE (Class {target_class})", fontweight="bold", fontsize=14)
    plt.tight_layout(rect=[0, 0, 1, 0.95])

    out_base = f"pdp_ice_{name.replace(' ', '_').lower()}"
    plt.savefig(os.path.join(OUT_DIR, f"{out_base}.png"))
    plt.savefig(os.path.join(OUT_DIR, f"{out_base}.pdf"))
    plt.close()
    print(f"✅ Saved combined PDP + ICE figure for {name}")

print("\n✅ All combined PDP + ICE figures saved in:", OUT_DIR)

⚠️ Dropping 't' to avoid leakage in PDPs.

=== Generating PDP + ICE for Random Forest ===
Top features for PDP/ICE: ['Ia', 'Ic', 'Ib', 'Va', 'Vb', 'Vc']
Plotting PDP/ICE for target class = 0
✅ Saved combined PDP + ICE figure for Random Forest

=== Generating PDP + ICE for XGBoost ===
Top features for PDP/ICE: ['Ib', 'Ia', 'Ic', 'Vb', 'Vc', 'Va']
Plotting PDP/ICE for target class = 0
⚠️ PDP/ICE failed for Ib: The 'estimator' parameter of partial_dependence must be an object implementing 'fit' and 'predict', an object implementing 'fit' and 'predict_proba' or an object implementing 'fit' and 'decision_function'. Got StandardScaler() instead.
⚠️ PDP/ICE failed for Ia: The 'estimator' parameter of partial_dependence must be an object implementing 'fit' and 'predict', an object implementing 'fit' and 'predict_proba' or an object implementing 'fit' and 'decision_function'. Got StandardScaler() instead.
⚠️ PDP/ICE failed for Ic: The 'estimator' parameter of partial_dependence must be an objec